In [30]:
pip install --upgrade google-cloud-language

In [2]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types


import pandas as pd
import numpy as np


In [ ]:
#def abc():
 #   client = language.LanguageServiceClient.from_service_account_json('/Users/jenniferwu/Downloads/second.json')


In [3]:
client = language.LanguageServiceClient.from_service_account_json('/Users/jenniferwu/Downloads/second.json')

# The text to analyze
text = u'''i am so pissed at this dog'''
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
#print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))
print('Sentiment: {}'.format(sentiment.score))

Text: i am so pissed at this dog
Sentiment: -0.6000000238418579


In [4]:
score = []

In [5]:
def score_range(score):
        if score <= -.8:
            o = -.8
        elif score > -.8 and score <= -.5:
            o = -.5
        elif score > -.5 and score <= -.3:
            o = -.3
        elif score > -.3 and score <= .3:
            o = 0
        elif score > .3 and score <= .5:
            o = .3
        elif score > .5 and score <= .8:
            o = .5
        else:
            o = .8
        return(o)

In [6]:
score_range(-.545)

-0.5

In [7]:
tweet = pd.read_csv('/Users/jenniferwu/Desktop/elonmusk_tweets.csv')

In [54]:
tweet.head()

id           created_at  \
0  849636868052275200  2017-04-05 14:56:29   
1  848988730585096192  2017-04-03 20:01:01   
2  848943072423497728  2017-04-03 16:59:35   
3  848935705057280001  2017-04-03 16:30:19   
4  848416049573658624  2017-04-02 06:05:23   

                                                text       date  year month  \
0  b'And so the robots spared humanity ... https:... 2017-04-05  2017    04   
1  b"@ForIn2020 @waltmossberg @mims @defcon_5 Exa... 2017-04-03  2017    04   
2      b'@waltmossberg @mims @defcon_5 Et tu, Walt?' 2017-04-03  2017    04   
3                b'Stormy weather in Shortville ...' 2017-04-03  2017    04   
4  b"@DaveLeeBBC @verge Coal is dying due to nat ... 2017-04-02  2017    04   

  day  score   source    cid  
0  05    0.1  twitter  18839  
1  03    0.1  twitter  18839  
2  03    0.0  twitter  18839  
3  03    0.1  twitter  18839  
4  02   -0.2  twitter  18839

In [55]:
tweet['date'] = tweet.created_at.str.partition(' ')[0]
tweet['date'] = pd.to_datetime(tweet.date, format="%Y-%m-%d")
tweet[['year','month','day']] = tweet.date.apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))

In [56]:
tweet = tweet[tweet.year=='2017']

tweet.shape

(377, 10)

In [57]:
score = []
for i in tweet.text:
        text = i
        document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
        sentiment = client.analyze_sentiment(document=document).document_sentiment
        score.append(sentiment.score)

In [58]:
tweet['score']=(score)

In [59]:
output = tweet.loc[:,["id","date","score"]]

In [60]:
output.head()

id       date  score
0  849636868052275200 2017-04-05    0.1
1  848988730585096192 2017-04-03    0.1
2  848943072423497728 2017-04-03    0.0
3  848935705057280001 2017-04-03    0.1
4  848416049573658624 2017-04-02   -0.2

In [61]:
tweet['score']=(score)
tweet['source']=["twitter"]*len(tweet)
tweet['cid']=['18839']*len(tweet)

In [62]:
output = tweet.loc[:,["cid","id","date","source","score"]]

In [63]:
import random

t_score = output[['cid','id', 'date', 'source', 'score']]
v_score = t_score
v_score.source = 'spotify'

for i in range(v_score.shape[0]):
    v_score.score[i] = random.uniform(-1,1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [64]:
output = pd.concat([output,v_score],axis = 0)

In [65]:
output['score_range'] = output.score.apply(score_range)

In [66]:
output.head()

cid                  id       date   source  score  score_range
0  18839  849636868052275200 2017-04-05  twitter    0.1          0.0
1  18839  848988730585096192 2017-04-03  twitter    0.1          0.0
2  18839  848943072423497728 2017-04-03  twitter    0.0          0.0
3  18839  848935705057280001 2017-04-03  twitter    0.1          0.0
4  18839  848416049573658624 2017-04-02  twitter   -0.2          0.0

In [94]:
from scipy import stats

#output.groupby(['date','cid']).agg(lambda x: stats.mode(x['score_range'])[0])

output_1 = output.groupby(['date','cid'])['score_range'].agg(pd.Series.mode)



In [96]:
output = output_1.reset_index(level=['cid','date'])

In [102]:
output['score_range'] = output.score_range.apply(np.mean)


In [103]:
output.head()

date    cid  score_range
0 2017-01-05  18839         0.00
1 2017-01-06  18839        -0.25
2 2017-01-07  18839         0.00
3 2017-01-08  18839         0.50
4 2017-01-10  18839         0.30

In [38]:
pip install plotly

     |████████████████████████████████| 7.1MB 2.0MB/s eta 0:00:01     |███████████████████████████▊    | 6.1MB 3.1MB/s eta 0:00:01
  Running setup.py install for retrying ... done
Note: you may need to restart the kernel to use updated packages.


In [104]:
import plotly.express as px

In [105]:
fig = px.line(output, x = 'date', y = 'score_range')

In [106]:
fig.show()

In [174]:
d = [{'id': '1183223595008266241', 'date': 'Sun Oct 13 03:31:02 +0000 2019', 'text': '@jay11w Recommended for sure. Both movies are incredible.'}, {'id': '1183222226130046980', 'date': 'Sun Oct 13 03:25:36 +0000 2019', 'text': 'First will rewatch Godfather Part II, then Space Jam'}, {'id': '1183221390016536581', 'date': 'Sun Oct 13 03:22:17 +0000 2019', 'text': 'Underappreciated 🎥, will watch!'}, {'id': '1183220879951462400', 'date': 'Sun Oct 13 03:20:15 +0000 2019', 'text': 'Is Space Jam good?'}, {'id': '1183201528036786178', 'date': 'Sun Oct 13 02:03:21 +0000 2019', 'text': '@Nobody_Really7 @anonyx10 @mayemusk So it goes'}, {'id': '1183198657136324610', 'date': 'Sun Oct 13 01:51:57 +0000 2019', 'text': '@anonyx10 @mayemusk Sure looks that way'}, {'id': '1183152738382667777', 'date': 'Sat Oct 12 22:49:29 +0000 2019', 'text': '@owlmaster08 @Tesla Nice 🤣🤣'}, {'id': '1183150916473516032', 'date': 'Sat Oct 12 22:42:15 +0000 2019', 'text': '@Erdayastronaut Solar'}, {'id': '1183150883162349569', 'date': 'Sat Oct 12 22:42:07 +0000 2019', 'text': '@Erdayastronaut Depends on total system efficiency &amp; how long the propellant plant can run to refill Starship, so 1 to 10MW as a rough guess'}, {'id': '1183123445787185152', 'date': 'Sat Oct 12 20:53:05 +0000 2019', 'text': '@bonnienorman Whoa, that does look good!'}, {'id': '1183119389211549696', 'date': 'Sat Oct 12 20:36:58 +0000 2019', 'text': '@Macwars Fair point. Will hold on EU price change until Tesla gets clarity from regulators about Smart Summon feature release date.'}, {'id': '1183118245458432001', 'date': 'Sat Oct 12 20:32:25 +0000 2019', 'text': 'French fries 🍟 are the devil 😈 https://t.co/AR3h57RPIA'}, {'id': '1183114722989621248', 'date': 'Sat Oct 12 20:18:25 +0000 2019', 'text': '@tesla_truth @BarkMSmeagol @vincent13031925 @jchybow @JohnnaCrider1 @28delayslater @flcnhvy @AfMusk .… https://t.co/vplI7NQDb6'}, {'id': '1183106801769697280', 'date': 'Sat Oct 12 19:46:57 +0000 2019', 'text': '@tesla_truth @BarkMSmeagol @vincent13031925 @jchybow @JohnnaCrider1 @28delayslater @flcnhvy @AfMusk Super messed up situation!'}, {'id': '1183082764515414016', 'date': 'Sat Oct 12 18:11:26 +0000 2019', 'text': '@cleantechnica Several finesse improvements coming to Smart Summon in coming weeks. Will be smooth as silk.'}, {'id': '1183081171644628992', 'date': 'Sat Oct 12 18:05:06 +0000 2019', 'text': '@wk057 @Tesla Should be much better at this point'}, {'id': '1183033425529147392', 'date': 'Sat Oct 12 14:55:23 +0000 2019', 'text': 'Nevada has an additional $3000 rebate for battery storage with solar https://t.co/bRibVqomzy'}, {'id': '1182828744748044288', 'date': 'Sat Oct 12 01:22:03 +0000 2019', 'text': 'Price for fully installed solar plus battery has been reduced ~10% to help those affected by blackouts'}, {'id': '1182828388852977665', 'date': 'Sat Oct 12 01:20:38 +0000 2019', 'text': '@brockm Yes, but probably a year or two away. Lot of fish to fry …'}, {'id': '1182828056416641025', 'date': 'Sat Oct 12 01:19:19 +0000 2019', 'text': '@bruce_mittleman Tesla cannot continue to lose money. To achieve our goal of environmental sustainability, Tesla mu… https://t.co/ArWGP4U79u'}, {'id': '1182827574361124864', 'date': 'Sat Oct 12 01:17:24 +0000 2019', 'text': 'Allows homeowner potential to go completely off grid'}, {'id': '1182827425551380482', 'date': 'Sat Oct 12 01:16:48 +0000 2019', 'text': 'V important: pure solar subscription is guaranteed to be a net money-maker for the homeowner or Tesla will take the… https://t.co/m3Z9eRHGzV'}, {'id': '1182826581070245888', 'date': 'Sat Oct 12 01:13:27 +0000 2019', 'text': '@cutza7 When the car is FSD without supervision, ie robotaxi, you’ll be able to earn far more than monthly lease/lo… https://t.co/0wyhUAyfnu'}, {'id': '1182825609686241280', 'date': 'Sat Oct 12 01:09:35 +0000 2019', 'text': '@tesla_addicted @sanjeev_r11 @vincent13031925 Putting panels on east or west facing roofs works well too. Energy lo… https://t.co/wG45nxMkNw'}, {'id': '1182824968200937472', 'date': 'Sat Oct 12 01:07:02 +0000 2019', 'text': '@wilsonlam 😉'}, {'id': '1182824823275155456', 'date': 'Sat Oct 12 01:06:28 +0000 2019', 'text': '@annerajb @Thebault_Arthur Working with engineering team to figure out best way to do upgrade without crushing serv… https://t.co/vENrP11kKA'}, {'id': '1182824078756872192', 'date': 'Sat Oct 12 01:03:30 +0000 2019', 'text': '@Thebault_Arthur Still working with RDW, the car regulatory authority in Netherlands. Unclear why there’s a delay.… https://t.co/9IwK1HoUs5'}, {'id': '1182823556830253056', 'date': 'Sat Oct 12 01:01:26 +0000 2019', 'text': 'Now that Tesla V10.0 with Smart Summon is out, Full Self-Driving price will increase by $1000 on Nov 1'}, {'id': '1182812717083611138', 'date': 'Sat Oct 12 00:18:22 +0000 2019', 'text': 'Get Tesla Solar plus Powerwall battery for 24/7 clean power &amp; no more blackouts! https://t.co/mDoPO17YB9'}, {'id': '1182808205425922049', 'date': 'Sat Oct 12 00:00:26 +0000 2019', 'text': '@mandeepdalip @Tesla Will be throughout Canada'}, {'id': '1182711457156190208', 'date': 'Fri Oct 11 17:35:59 +0000 2019', 'text': '@vincent13031925 It’s something special'}, {'id': '1182327145793634304', 'date': 'Thu Oct 10 16:08:52 +0000 2019', 'text': '@Gfilche @Tesla Fundamental metric of good of Tesla is how many years sooner it brings the advent of a sustainable energy future'}, {'id': '1182192515778170881', 'date': 'Thu Oct 10 07:13:54 +0000 2019', 'text': '@RationalEtienne @tesla_truth @AfMusk @28delayslater @EvaFoxU @JohnnaCrider1 @vincent13031925 @mayemusk… https://t.co/Tk7pCpZz6X'}, {'id': '1182189869344247809', 'date': 'Thu Oct 10 07:03:23 +0000 2019', 'text': '@sara_boutall @In2867Muk @parisreview I ♥️ Asterix &amp; Obelix!'}, {'id': '1182189464736526337', 'date': 'Thu Oct 10 07:01:47 +0000 2019', 'text': '@tesla_truth Exactly'}, {'id': '1182188397965656064', 'date': 'Thu Oct 10 06:57:32 +0000 2019', 'text': '@Milostoessel @Tesla @TeslaSchweiz Ok'}, {'id': '1182089703039700993', 'date': 'Thu Oct 10 00:25:22 +0000 2019', 'text': 'Also adding Tesla Solar to our Supercharger stations as fast as possible. Goal is 24/7 clean power with no blackouts.'}, {'id': '1182089226826813440', 'date': 'Thu Oct 10 00:23:28 +0000 2019', 'text': 'All Tesla Supercharger stations in regions affected by California power outages will have Tesla Powerpacks within n… https://t.co/WK81oHrGqj'}, {'id': '1182061315390492673', 'date': 'Wed Oct 09 22:32:33 +0000 2019', 'text': 'Sooo annoying!! https://t.co/QKtCC0YgDG'}, {'id': '1182059866052952066', 'date': 'Wed Oct 09 22:26:48 +0000 2019', 'text': 'https://t.co/BkKeiIzNPM'}, {'id': '1182005812975128576', 'date': 'Wed Oct 09 18:52:01 +0000 2019', 'text': '@Teslarati We’re changing Powerwall Storm Watch to Disaster Watch. Covers more bases.'}, {'id': '1181991353212817408', 'date': 'Wed Oct 09 17:54:33 +0000 2019', 'text': '@Kristennetten @annerajb @tesla_truth 🚀🚘 🧠🕳'}, {'id': '1181990505363599360', 'date': 'Wed Oct 09 17:51:11 +0000 2019', 'text': '@annerajb @tesla_truth Yes'}, {'id': '1181990398379483137', 'date': 'Wed Oct 09 17:50:45 +0000 2019', 'text': '@tesla_truth No change'}, {'id': '1181990010657984513', 'date': 'Wed Oct 09 17:49:13 +0000 2019', 'text': '@JimBridenstine @NASAMoon Important for Moonbase Alpha!'}, {'id': '1181989324797009922', 'date': 'Wed Oct 09 17:46:29 +0000 2019', 'text': '@GeorgKonjovic Working on it'}, {'id': '1181989103606255616', 'date': 'Wed Oct 09 17:45:37 +0000 2019', 'text': '@_GaryTheBot Even Gary likes it!'}, {'id': '1181988619000500224', 'date': 'Wed Oct 09 17:43:41 +0000 2019', 'text': 'It’s really quite fun!'}, {'id': '1181988380990558208', 'date': 'Wed Oct 09 17:42:44 +0000 2019', 'text': 'But we call it “caraoke”, because it’s in a 🚘'}, {'id': '1181988180326633472', 'date': 'Wed Oct 09 17:41:57 +0000 2019', 'text': 'Tesla software V10.0 comes with karaoke 🎤'}, {'id': '1181987789589508096', 'date': 'Wed Oct 09 17:40:23 +0000 2019', 'text': '@nextspaceflight Yes'}, {'id': '1181987612992524288', 'date': 'Wed Oct 09 17:39:41 +0000 2019', 'text': 'A Shortfall of Gravitas'}, {'id': '1181980074037301248', 'date': 'Wed Oct 09 17:09:44 +0000 2019', 'text': '@Thebault_Arthur @_GaryTheBot Next update'}, {'id': '1181979746550202368', 'date': 'Wed Oct 09 17:08:26 +0000 2019', 'text': '@_GaryTheBot siq burn, Gary, siq burn'}, {'id': '1181979597467873280', 'date': 'Wed Oct 09 17:07:50 +0000 2019', 'text': '🦌'}, {'id': '1181979201915604993', 'date': 'Wed Oct 09 17:06:16 +0000 2019', 'text': 'Why is there no emoji for dignity!!'}, {'id': '1181841203286790144', 'date': 'Wed Oct 09 07:57:55 +0000 2019', 'text': '@teslaownersSV Yes'}, {'id': '1181837821855711232', 'date': 'Wed Oct 09 07:44:28 +0000 2019', 'text': '@sara_boutall Door hinge'}, {'id': '1181837097444896768', 'date': 'Wed Oct 09 07:41:36 +0000 2019', 'text': 'the color orange is named after the fruit'}, {'id': '1181781720841908224', 'date': 'Wed Oct 09 04:01:33 +0000 2019', 'text': '@alex_gayer @teslaownersSV Will remove'}, {'id': '1181781181316026369', 'date': 'Wed Oct 09 03:59:24 +0000 2019', 'text': '@teslaownersSV Thanks for mentioning. Should be enabled on all loaner cars.'}, {'id': '1181781021785653248', 'date': 'Wed Oct 09 03:58:46 +0000 2019', 'text': '@Aero_JohnG @Erdayastronaut @SciGuySpace Yeah'}, {'id': '1181738919076851712', 'date': 'Wed Oct 09 01:11:28 +0000 2019', 'text': '@annerajb @Teslarati Not easily, as requires wire harness changes'}, {'id': '1181738665963208704', 'date': 'Wed Oct 09 01:10:28 +0000 2019', 'text': '@Teslarati 🦶 💨'}, {'id': '1181658171598917632', 'date': 'Tue Oct 08 19:50:36 +0000 2019', 'text': '@justpaulinelol I’m in the credits'}, {'id': '1181656686236508160', 'date': 'Tue Oct 08 19:44:42 +0000 2019', 'text': 'Worth reading “Human Compatible” by Stuart Russell (he’s great!) about future AI risks &amp; solutions https://t.co/ZCdvZrrcVf'}, {'id': '1181585837869297664', 'date': 'Tue Oct 08 15:03:11 +0000 2019', 'text': '@flcnhvy @SciGuySpace All hardware is at the Cape. Need to do static fire and reconfigure for flight. Launch probably late Nov / early Dec.'}, {'id': '1181584415362707456', 'date': 'Tue Oct 08 14:57:32 +0000 2019', 'text': '@SciGuySpace We had to reallocate some resources to speed this up &amp; received great support from Airborne, our parac… https://t.co/nuM43KoKDC'}, {'id': '1181579173388673025', 'date': 'Tue Oct 08 14:36:42 +0000 2019', 'text': '@SciGuySpace For what it’s worth, the SpaceX schedule, which I’ve just reviewed in depth, shows Falcon &amp; Dragon at… https://t.co/AloXFnfr3W'}, {'id': '1181497517759811584', 'date': 'Tue Oct 08 09:12:14 +0000 2019', 'text': '@alex_gayer 🤣'}, {'id': '1181345349148917760', 'date': 'Mon Oct 07 23:07:34 +0000 2019', 'text': '@flcnhvy @BnOrdhaug @Gizmodo Culture Class is really something'}, {'id': '1181344603716255745', 'date': 'Mon Oct 07 23:04:36 +0000 2019', 'text': '@BnOrdhaug @Gizmodo I hope so'}, {'id': '1181343438630838273', 'date': 'Mon Oct 07 22:59:58 +0000 2019', 'text': '@Gizmodo We’re doomed!'}, {'id': '1181304116498165760', 'date': 'Mon Oct 07 20:23:43 +0000 2019', 'text': '@pewdiepie Hāhā'}, {'id': '1181295442132230144', 'date': 'Mon Oct 07 19:49:15 +0000 2019', 'text': '@browserdotsys Haha true'}, {'id': '1181114635962814464', 'date': 'Mon Oct 07 07:50:47 +0000 2019', 'text': '@WellsAdams Done'}, {'id': '1181087900051492864', 'date': 'Mon Oct 07 06:04:33 +0000 2019', 'text': 'Trouble! https://t.co/qo2hR7Vcsp'}, {'id': '1181083687384838145', 'date': 'Mon Oct 07 05:47:49 +0000 2019', 'text': 'And that’s just the tip of the iceberg https://t.co/cgRKiDsdbK'}, {'id': '1181070766101549056', 'date': 'Mon Oct 07 04:56:28 +0000 2019', 'text': '@jesterpb @yukaichou @Tesla Agreed'}, {'id': '1180957431368568832', 'date': 'Sun Oct 06 21:26:07 +0000 2019', 'text': '@TameemJi @yukaichou @Tesla Good ones'}, {'id': '1180928986903265280', 'date': 'Sun Oct 06 19:33:05 +0000 2019', 'text': '@SpodieDodie Agreed, fair point'}, {'id': '1180927887970422785', 'date': 'Sun Oct 06 19:28:43 +0000 2019', 'text': '@yukaichou @Tesla Any feature requests?'}, {'id': '1180927745301176320', 'date': 'Sun Oct 06 19:28:09 +0000 2019', 'text': 'Given how quickly the world is shifting to electric vehicles, a gas/diesel vehicle bought today will probably have… https://t.co/aPRBK7hr1y'}, {'id': '1180878496224038914', 'date': 'Sun Oct 06 16:12:27 +0000 2019', 'text': '💨 &amp; 🐐 sounds too (also, of course)'}, {'id': '1180877799734665216', 'date': 'Sun Oct 06 16:09:41 +0000 2019', 'text': '@JudaMeg Will consider'}, {'id': '1180877420821237760', 'date': 'Sun Oct 06 16:08:11 +0000 2019', 'text': '@teslaownersSV Sure'}, {'id': '1180877114226008064', 'date': 'Sun Oct 06 16:06:58 +0000 2019', 'text': 'Customized horn &amp; movement sounds (coconuts being one, of course) coming to Teslas soon'}, {'id': '1180569585558228992', 'date': 'Sat Oct 05 19:44:57 +0000 2019', 'text': '@Erdayastronaut @bluemoondance74 @EvaFoxU Haha'}, {'id': '1180553336082157568', 'date': 'Sat Oct 05 18:40:23 +0000 2019', 'text': '@bluemoondance74 @EvaFoxU We will understand the next question'}, {'id': '1180553063301468160', 'date': 'Sat Oct 05 18:39:18 +0000 2019', 'text': '@justpaulinelol @EvaFoxU True'}, {'id': '1180552829498322945', 'date': 'Sat Oct 05 18:38:22 +0000 2019', 'text': '@EvaFoxU To understand the nature of consciousness'}, {'id': '1180320978435543042', 'date': 'Sat Oct 05 03:17:05 +0000 2019', 'text': '@Mikeado @Lancelot_Fox @Tommymcglynn @trevorandersen @fermatslibrary It’s true 🤣🤣'}, {'id': '1180264594364870660', 'date': 'Fri Oct 04 23:33:02 +0000 2019', 'text': '@vincent13031925 @Sofiaan @SCE Exactly'}, {'id': '1180165032690233345', 'date': 'Fri Oct 04 16:57:24 +0000 2019', 'text': '@rklau Great list of hotel hacks in this thread!'}, {'id': '1179996203574013953', 'date': 'Fri Oct 04 05:46:32 +0000 2019', 'text': '@tesla_truth 🤣🤣'}, {'id': '1179957355628253185', 'date': 'Fri Oct 04 03:12:10 +0000 2019', 'text': 'Excited about launching @NASA asteroid defense mission! #Armageddon69 https://t.co/6Hz4CKOLwU'}, {'id': '1179941711306358787', 'date': 'Fri Oct 04 02:10:00 +0000 2019', 'text': '@nichegamer Let’s go boyz!'}, {'id': '1179925839669891072', 'date': 'Fri Oct 04 01:06:56 +0000 2019', 'text': '@TeslaOwnersSK @Tesla Sorry it took so long!'}, {'id': '1179897546161848320', 'date': 'Thu Oct 03 23:14:31 +0000 2019', 'text': '@TarikCelebii Wow, cool! Summon will get steadily smoother until it’s like an expert chauffeur who knows you well.'}, {'id': '1179799146464628736', 'date': 'Thu Oct 03 16:43:30 +0000 2019', 'text': '@flcnhvy @HarryStoltz1 Welded steel'}, {'id': '1179624066375774209', 'date': 'Thu Oct 03 05:07:48 +0000 2019', 'text': '@HarryStoltz1 Probably Starlink 🛰 &amp; some fun things'}, {'id': '1179576466721259520', 'date': 'Thu Oct 03 01:58:39 +0000 2019', 'text': 'Rosetta by Vangelis is incredible https://t.co/MKqbsdVVsN'}, {'id': '1179524946478817280', 'date': 'Wed Oct 02 22:33:56 +0000 2019', 'text': '@teslaownersSV @arctechinc Not sure, will ask regulators'}, {'id': '1179524799804002304', 'date': 'Wed Oct 02 22:33:21 +0000 2019', 'text': '@AdamTForbes @annerajb @flcnhvy Enhanced should get it too'}, {'id': '1179524256792641542', 'date': 'Wed Oct 02 22:31:11 +0000 2019', 'text': '@arctechinc Aiming for this weekend!'}, {'id': '1179523171780681728', 'date': 'Wed Oct 02 22:26:53 +0000 2019', 'text': '@annerajb @flcnhvy Hopefully, a month or two, although it may be a little silly at first'}, {'id': '1179522786240319488', 'date': 'Wed Oct 02 22:25:21 +0000 2019', 'text': '@flcnhvy Not yet'}, {'id': '1179522738278395905', 'date': 'Wed Oct 02 22:25:09 +0000 2019', 'text': '@MemesOfMars First point release update expected to go out this weekend'}, {'id': '1179520622004588544', 'date': 'Wed Oct 02 22:16:45 +0000 2019', 'text': 'Over 550,000 Tesla Smart Summon uses in first few days!'}, {'id': '1179519667808768000', 'date': 'Wed Oct 02 22:12:57 +0000 2019', 'text': '@tesla_truth Thanks, Tesla team is the strongest it’s ever been &amp; doing an amazing job! Honored to work beside them.'}, {'id': '1179159128708538370', 'date': 'Tue Oct 01 22:20:18 +0000 2019', 'text': '@justpaulinelol @KeithMansfield @Erdayastronaut If 20km works, then orbit'}, {'id': '1179158176618315776', 'date': 'Tue Oct 01 22:16:31 +0000 2019', 'text': '@KeithMansfield @Erdayastronaut Great summary of the key points!'}, {'id': '1179154111616274433', 'date': 'Tue Oct 01 22:00:22 +0000 2019', 'text': '@OranMaliphant @Erdayastronaut @AngelNDevil2 @MarkChica @FutureJurvetson @bluemoondance74 @SpaceX That’s about righ… https://t.co/UuXAf1fpBh'}, {'id': '1179107730876796928', 'date': 'Tue Oct 01 18:56:04 +0000 2019', 'text': '@AngelNDevil2 @MarkChica @FutureJurvetson @bluemoondance74 @SpaceX It’s all the “secondary” structure that concerns… https://t.co/vQeVrk8J4V'}, {'id': '1179107539352313856', 'date': 'Tue Oct 01 18:55:18 +0000 2019', 'text': '@AngelNDevil2 @MarkChica @FutureJurvetson @bluemoondance74 @SpaceX Raptor cost is tracking to well under $1M for V1… https://t.co/ic6sKxZLFG'}, {'id': '1179090138644668416', 'date': 'Tue Oct 01 17:46:10 +0000 2019', 'text': '@JohnnaCrider1 @RationalEtienne @LaurenRow5 @RenataKonkoly @Kristennetten @EvaFoxU @MemesOfMars @PositivePyrami1 Stop sending me this 💩 🤣🤣'}, {'id': '1179079219801812993', 'date': 'Tue Oct 01 17:02:46 +0000 2019', 'text': '@RationalEtienne @LaurenRow5 @EvaFoxU @Kristennetten @SteveHamel16 @TheElonMasked @MemesOfMars @MichellBasler… https://t.co/O4xgr8y6kc'}, {'id': '1179078946941353984', 'date': 'Tue Oct 01 17:01:41 +0000 2019', 'text': '@Kristennetten @SS4Adrian I have that exact thought every time I get the flu'}, {'id': '1179074198540435456', 'date': 'Tue Oct 01 16:42:49 +0000 2019', 'text': '@SS4Adrian Clearly a sign of the apocalypse!'}, {'id': '1178956709731024896', 'date': 'Tue Oct 01 08:55:58 +0000 2019', 'text': '@engineers_feed Lot of ways to melt &amp; join steel. Mostly just needs to be welded consistently &amp; with precise parts.'}, {'id': '1178956134968807424', 'date': 'Tue Oct 01 08:53:41 +0000 2019', 'text': '@engineers_feed Thankfully not. FSW is very difficult to get right &amp; not needed with steel.'}, {'id': '1178938112195514368', 'date': 'Tue Oct 01 07:42:04 +0000 2019', 'text': 'Production version will be a lot more polished than this prototype, but still fun to see'}, {'id': '1178934984167448577', 'date': 'Tue Oct 01 07:29:38 +0000 2019', 'text': '@ColinPearceUK Bigger flaps/fins at rear also moves center of pressure back'}, {'id': '1178932681028980736', 'date': 'Tue Oct 01 07:20:29 +0000 2019', 'text': '@TrevorMahlmann Haha yes'}, {'id': '1178932062733996032', 'date': 'Tue Oct 01 07:18:01 +0000 2019', 'text': '@HarryStoltz1 In solving for a good Mars climate, we will learn a great deal about how to do so on Earth. It is the inverse problem.'}, {'id': '1178931253229187072', 'date': 'Tue Oct 01 07:14:48 +0000 2019', 'text': 'Inside Starship cargo bay. Header tanks mounted in tip of nosecone to offset engine weight at rear. https://t.co/EJSwqMCooA'}, {'id': '1178905464202883072', 'date': 'Tue Oct 01 05:32:20 +0000 2019', 'text': '@EvaFoxU @DisruptResearch @Tesla @karpathy Cool video!'}, {'id': '1178818555061493761', 'date': 'Mon Sep 30 23:46:59 +0000 2019', 'text': '@DisruptResearch @Tesla @karpathy Probably our most viral feature ever. Most of the Autopilot team has been working… https://t.co/Vs7de166X3'}, {'id': '1178807103273631744', 'date': 'Mon Sep 30 23:01:29 +0000 2019', 'text': '@S_Padival @Tesla @business @liamdenning @josheidelson @danahull @ZekeFaux Good question'}, {'id': '1178107012141535232', 'date': 'Sun Sep 29 00:39:34 +0000 2019', 'text': '@thejackbeyer 😀'}, {'id': '1178014342031298561', 'date': 'Sat Sep 28 18:31:20 +0000 2019', 'text': 'Rage, rage against the dying of the light https://t.co/aGRw4VowAQ'}, {'id': '1178013709546991619', 'date': 'Sat Sep 28 18:28:49 +0000 2019', 'text': '@tesla_truth @NRF_designs We’re going to use the pedestrian pink noise speaker to talk to people &amp; create customize… https://t.co/cWLXua7AnI'}, {'id': '1178012340383621120', 'date': 'Sat Sep 28 18:23:23 +0000 2019', 'text': '@tesla_truth Special light pattern (but not hazards, as they’re too shrill) seems right'}, {'id': '1178011464852004864', 'date': 'Sat Sep 28 18:19:54 +0000 2019', 'text': '@tesla_truth Makes sense'}, {'id': '1177961429703507968', 'date': 'Sat Sep 28 15:01:04 +0000 2019', 'text': '@tesla_truth @Tesla Precisely'}, {'id': '1177665592242794497', 'date': 'Fri Sep 27 19:25:31 +0000 2019', 'text': 'To make life as we know it multiplanetary'}, {'id': '1177665504699224064', 'date': 'Fri Sep 27 19:25:10 +0000 2019', 'text': 'Starship will allow us to inhabit other worlds'}, {'id': '1177662806117584896', 'date': 'Fri Sep 27 19:14:27 +0000 2019', 'text': 'Starship halves being joined https://t.co/7pdxfh8e81'}, {'id': '1177657997134221312', 'date': 'Fri Sep 27 18:55:21 +0000 2019', 'text': 'Summon is improving rapidly \nhttps://t.co/khgXDRnD2Z'}, {'id': '1177631358987849729', 'date': 'Fri Sep 27 17:09:29 +0000 2019', 'text': '@flcnhvy @Erdayastronaut @Cor_SPACE Just SX500 so far, but we use many superalloys developed by others, eg Inco 718… https://t.co/2ZRmiA1peV'}, {'id': '1177630155801419778', 'date': 'Fri Sep 27 17:04:43 +0000 2019', 'text': '@Erdayastronaut @Cor_SPACE There’s so much power going through this engine that it’s close to the limit of the know… https://t.co/7LEcDvIRi0'}, {'id': '1177628992762855426', 'date': 'Fri Sep 27 17:00:05 +0000 2019', 'text': '@Carter__99 @Erdayastronaut @askingbubble385 @Cor_SPACE Yes'}, {'id': '1177628790609997825', 'date': 'Fri Sep 27 16:59:17 +0000 2019', 'text': '@Erdayastronaut @Cor_SPACE Well, hopefully closer to 300 bar over time. However, that the chamber/nozzle jacket has… https://t.co/VWegWXxxsL'}, {'id': '1177622466727821312', 'date': 'Fri Sep 27 16:34:09 +0000 2019', 'text': '@flcnhvy @Erdayastronaut @Cor_SPACE No, we’ve been focusing on production (now on SN12). Only need 250 bar or even… https://t.co/MCIxOMAXeJ'}, {'id': '1177621843219365888', 'date': 'Fri Sep 27 16:31:41 +0000 2019', 'text': '@NCBirbhan @Erdayastronaut @Cor_SPACE Probably around 6 or 7pm'}, {'id': '1177621104686329856', 'date': 'Fri Sep 27 16:28:45 +0000 2019', 'text': '@Erdayastronaut @Cor_SPACE What blows my mind is that peak pressure in regen chamber &amp; nozzle is *far* above combus… https://t.co/QZf6knzhq9'}, {'id': '1177618467136008194', 'date': 'Fri Sep 27 16:18:16 +0000 2019', 'text': '@GaryTheBot_ Figure of speech'}, {'id': '1177618401159602176', 'date': 'Fri Sep 27 16:18:00 +0000 2019', 'text': 'Meanwhile, back in reality https://t.co/NrHHB9G81L'}, {'id': '1177618032719319040', 'date': 'Fri Sep 27 16:16:32 +0000 2019', 'text': '@DLoesch Media report was false'}, {'id': '1177387141116002304', 'date': 'Fri Sep 27 00:59:03 +0000 2019', 'text': '@Erdayastronaut @Cor_SPACE Raptor uses milled copper channels with an inconel jacket all the way down'}, {'id': '1177314408604680192', 'date': 'Thu Sep 26 20:10:03 +0000 2019', 'text': 'Three Raptors on a Starship https://t.co/UrRiD62EVk'}, {'id': '1177259561117372417', 'date': 'Thu Sep 26 16:32:06 +0000 2019', 'text': '@EvaFoxU @BrendanCogbill7 Always'}, {'id': '1177241843093753862', 'date': 'Thu Sep 26 15:21:42 +0000 2019', 'text': '@SpacePadreIsle @SpaceX Wow, it’s been 5 years since we started at Boca!'}, {'id': '1177237783489863681', 'date': 'Thu Sep 26 15:05:34 +0000 2019', 'text': '@BrendanCogbill7 About a week. Will include slight improvements, based on initial user feedback.'}, {'id': '1177236839406559232', 'date': 'Thu Sep 26 15:01:49 +0000 2019', 'text': '@justpaulinelol We look for rare corner case issues with a limited release US build, then expand internationally as… https://t.co/Lqw9n2ZHvR'}, {'id': '1177235963405193216', 'date': 'Thu Sep 26 14:58:20 +0000 2019', 'text': '@GaryTheBot_ Gary, you rock!'}, {'id': '1177235772857970688', 'date': 'Thu Sep 26 14:57:34 +0000 2019', 'text': 'Tesla V10.0 just started rolling out to all US Tesla owners with FSD option &amp; “advanced download” selected in vehicle software settings'}, {'id': '1177101142980825093', 'date': 'Thu Sep 26 06:02:36 +0000 2019', 'text': '@moyo5150 More like https://t.co/f8BUlz0Xdo'}, {'id': '1177100271937212416', 'date': 'Thu Sep 26 05:59:08 +0000 2019', 'text': 'Anonymous bot swarms deserve a closer examination. If they’re evolving rapidly, something’s up.'}, {'id': '1177099375287271424', 'date': 'Thu Sep 26 05:55:35 +0000 2019', 'text': 'If advanced AI (beyond basic bots) hasn’t been applied to manipulate social media, it won’t be long before it is'}, {'id': '1177096273628778496', 'date': 'Thu Sep 26 05:43:15 +0000 2019', 'text': '@Wally_Wests Exactly'}, {'id': '1177090550916239360', 'date': 'Thu Sep 26 05:20:31 +0000 2019', 'text': '@Kristennetten @annerajb @Jennerator211 @Erdayastronaut That would be a nice touch'}, {'id': '1177085193007456256', 'date': 'Thu Sep 26 04:59:13 +0000 2019', 'text': '@flcnhvy @28delayslater @DJSnM This is quite a complex beast, but hopefully within a month or so'}, {'id': '1177084577128448000', 'date': 'Thu Sep 26 04:56:47 +0000 2019', 'text': '@28delayslater @DJSnM Haha awesome'}, {'id': '1177074975020220416', 'date': 'Thu Sep 26 04:18:37 +0000 2019', 'text': '@MarkChica @Kristennetten @TeslaGong @teslaownersSV @Teslatunity @Tesla We could probably do that'}, {'id': '1177070801121468418', 'date': 'Thu Sep 26 04:02:02 +0000 2019', 'text': '@Erdayastronaut 50m'}, {'id': '1177069611142901762', 'date': 'Thu Sep 26 03:57:18 +0000 2019', 'text': '@TeslaGong @teslaownersSV @Teslatunity @Tesla Not sure we could go quite that tight, but we can for sure allow much… https://t.co/wolYBBZMS7'}, {'id': '1177067257106747392', 'date': 'Thu Sep 26 03:47:57 +0000 2019', 'text': '@annerajb @Jennerator211 @Erdayastronaut With rare exceptions, composites would make Starship heavier. They don’t s… https://t.co/OeeigvDiI8'}, {'id': '1177066483375058944', 'date': 'Thu Sep 26 03:44:53 +0000 2019', 'text': '@Jennerator211 @Erdayastronaut Mk1 ship is around 200 tons dry &amp; 1400 tons wet, but aiming for 120 by Mk4 or Mk5. T… https://t.co/yTUpGx0Ppa'}, {'id': '1177064952429010945', 'date': 'Thu Sep 26 03:38:48 +0000 2019', 'text': '@teslaownersSV @Teslatunity @Tesla Absolutely'}, {'id': '1177062354577756160', 'date': 'Thu Sep 26 03:28:28 +0000 2019', 'text': '@Teslatunity @Tesla You’re welcome! Feature requests always appreciated.'}, {'id': '1177051800769138690', 'date': 'Thu Sep 26 02:46:32 +0000 2019', 'text': '@Erdayastronaut @kimitalvitie Six. Two windward, one under each fin &amp; two leeward. Provides redundancy for landing on unimproved surfaces.'}, {'id': '1177050841041125376', 'date': 'Thu Sep 26 02:42:43 +0000 2019', 'text': 'On the transporter https://t.co/CGNj1MlKAU'}, {'id': '1177049277375250434', 'date': 'Thu Sep 26 02:36:30 +0000 2019', 'text': 'Lifting lower section of Starship https://t.co/bfW17i469a'}, {'id': '1176936806928879616', 'date': 'Wed Sep 25 19:09:35 +0000 2019', 'text': '@justpaulinelol @EvaFoxU @katlinegrey https://t.co/IzVngWWY7G'}, {'id': '1176930839411642368', 'date': 'Wed Sep 25 18:45:53 +0000 2019', 'text': '@EvaFoxU @katlinegrey правда'}, {'id': '1176928113285971968', 'date': 'Wed Sep 25 18:35:03 +0000 2019', 'text': '@katlinegrey язык хорошо подвешен'}, {'id': '1176737065624453125', 'date': 'Wed Sep 25 05:55:53 +0000 2019', 'text': '@levie Exactly'}, {'id': '1176596320775094273', 'date': 'Tue Sep 24 20:36:37 +0000 2019', 'text': '@tannerfox Impressive work by Chad &amp; Steve!'}, {'id': '1176589331567452161', 'date': 'Tue Sep 24 20:08:51 +0000 2019', 'text': '@SamNazarius @HarryStoltz1 @justpaulinelol @Teslarati Almost entirely C++, but we did have an (advanced for the tim… https://t.co/0SbJqgtWsG'}, {'id': '1176588054670323713', 'date': 'Tue Sep 24 20:03:46 +0000 2019', 'text': '@HarryStoltz1 @justpaulinelol @Teslarati Turbo Pascal'}, {'id': '1176587698401947650', 'date': 'Tue Sep 24 20:02:21 +0000 2019', 'text': '@justpaulinelol @Teslarati Mostly C/C++, Python &amp; SQL'}, {'id': '1176586684751908872', 'date': 'Tue Sep 24 19:58:20 +0000 2019', 'text': '@flcnhvy @Erdayastronaut @Tesla It would be pretty embarrassing to use a non-Tesla battery!'}, {'id': '1176585904091275264', 'date': 'Tue Sep 24 19:55:14 +0000 2019', 'text': '@Teslarati This includes original engineering of new factory processes &amp; cars. Great engineers will only join if or… https://t.co/zCAndlhS72'}]

In [175]:
d= pd.DataFrame(d)

In [176]:
d.head()

id                            date  \
0  1183223595008266241  Sun Oct 13 03:31:02 +0000 2019   
1  1183222226130046980  Sun Oct 13 03:25:36 +0000 2019   
2  1183221390016536581  Sun Oct 13 03:22:17 +0000 2019   
3  1183220879951462400  Sun Oct 13 03:20:15 +0000 2019   
4  1183201528036786178  Sun Oct 13 02:03:21 +0000 2019   

                                                text  
0  @jay11w Recommended for sure. Both movies are ...  
1  First will rewatch Godfather Part II, then Spa...  
2                    Underappreciated 🎥, will watch!  
3                                 Is Space Jam good?  
4     @Nobody_Really7 @anonyx10 @mayemusk So it goes

In [204]:
from dateutil.parser import parse
from datetime import datetime

d = [{'id': '1183223595008266241', 'date': 'Sun Oct 13 03:31:02 +0000 2019', 'text': '@jay11w Recommended for sure. Both movies are incredible.'}, {'id': '1183222226130046980', 'date': 'Sun Oct 13 03:25:36 +0000 2019', 'text': 'First will rewatch Godfather Part II, then Space Jam'}, {'id': '1183221390016536581', 'date': 'Sun Oct 13 03:22:17 +0000 2019', 'text': 'Underappreciated 🎥, will watch!'}, {'id': '1183220879951462400', 'date': 'Sun Oct 13 03:20:15 +0000 2019', 'text': 'Is Space Jam good?'}, {'id': '1183201528036786178', 'date': 'Sun Oct 13 02:03:21 +0000 2019', 'text': '@Nobody_Really7 @anonyx10 @mayemusk So it goes'}, {'id': '1183198657136324610', 'date': 'Sun Oct 13 01:51:57 +0000 2019', 'text': '@anonyx10 @mayemusk Sure looks that way'}, {'id': '1183152738382667777', 'date': 'Sat Oct 12 22:49:29 +0000 2019', 'text': '@owlmaster08 @Tesla Nice 🤣🤣'}, {'id': '1183150916473516032', 'date': 'Sat Oct 12 22:42:15 +0000 2019', 'text': '@Erdayastronaut Solar'}, {'id': '1183150883162349569', 'date': 'Sat Oct 12 22:42:07 +0000 2019', 'text': '@Erdayastronaut Depends on total system efficiency &amp; how long the propellant plant can run to refill Starship, so 1 to 10MW as a rough guess'}, {'id': '1183123445787185152', 'date': 'Sat Oct 12 20:53:05 +0000 2019', 'text': '@bonnienorman Whoa, that does look good!'}, {'id': '1183119389211549696', 'date': 'Sat Oct 12 20:36:58 +0000 2019', 'text': '@Macwars Fair point. Will hold on EU price change until Tesla gets clarity from regulators about Smart Summon feature release date.'}, {'id': '1183118245458432001', 'date': 'Sat Oct 12 20:32:25 +0000 2019', 'text': 'French fries 🍟 are the devil 😈 https://t.co/AR3h57RPIA'}, {'id': '1183114722989621248', 'date': 'Sat Oct 12 20:18:25 +0000 2019', 'text': '@tesla_truth @BarkMSmeagol @vincent13031925 @jchybow @JohnnaCrider1 @28delayslater @flcnhvy @AfMusk .… https://t.co/vplI7NQDb6'}, {'id': '1183106801769697280', 'date': 'Sat Oct 12 19:46:57 +0000 2019', 'text': '@tesla_truth @BarkMSmeagol @vincent13031925 @jchybow @JohnnaCrider1 @28delayslater @flcnhvy @AfMusk Super messed up situation!'}, {'id': '1183082764515414016', 'date': 'Sat Oct 12 18:11:26 +0000 2019', 'text': '@cleantechnica Several finesse improvements coming to Smart Summon in coming weeks. Will be smooth as silk.'}, {'id': '1183081171644628992', 'date': 'Sat Oct 12 18:05:06 +0000 2019', 'text': '@wk057 @Tesla Should be much better at this point'}, {'id': '1183033425529147392', 'date': 'Sat Oct 12 14:55:23 +0000 2019', 'text': 'Nevada has an additional $3000 rebate for battery storage with solar https://t.co/bRibVqomzy'}, {'id': '1182828744748044288', 'date': 'Sat Oct 12 01:22:03 +0000 2019', 'text': 'Price for fully installed solar plus battery has been reduced ~10% to help those affected by blackouts'}, {'id': '1182828388852977665', 'date': 'Sat Oct 12 01:20:38 +0000 2019', 'text': '@brockm Yes, but probably a year or two away. Lot of fish to fry …'}, {'id': '1182828056416641025', 'date': 'Sat Oct 12 01:19:19 +0000 2019', 'text': '@bruce_mittleman Tesla cannot continue to lose money. To achieve our goal of environmental sustainability, Tesla mu… https://t.co/ArWGP4U79u'}, {'id': '1182827574361124864', 'date': 'Sat Oct 12 01:17:24 +0000 2019', 'text': 'Allows homeowner potential to go completely off grid'}, {'id': '1182827425551380482', 'date': 'Sat Oct 12 01:16:48 +0000 2019', 'text': 'V important: pure solar subscription is guaranteed to be a net money-maker for the homeowner or Tesla will take the… https://t.co/m3Z9eRHGzV'}, {'id': '1182826581070245888', 'date': 'Sat Oct 12 01:13:27 +0000 2019', 'text': '@cutza7 When the car is FSD without supervision, ie robotaxi, you’ll be able to earn far more than monthly lease/lo… https://t.co/0wyhUAyfnu'}, {'id': '1182825609686241280', 'date': 'Sat Oct 12 01:09:35 +0000 2019', 'text': '@tesla_addicted @sanjeev_r11 @vincent13031925 Putting panels on east or west facing roofs works well too. Energy lo… https://t.co/wG45nxMkNw'}, {'id': '1182824968200937472', 'date': 'Sat Oct 12 01:07:02 +0000 2019', 'text': '@wilsonlam 😉'}, {'id': '1182824823275155456', 'date': 'Sat Oct 12 01:06:28 +0000 2019', 'text': '@annerajb @Thebault_Arthur Working with engineering team to figure out best way to do upgrade without crushing serv… https://t.co/vENrP11kKA'}, {'id': '1182824078756872192', 'date': 'Sat Oct 12 01:03:30 +0000 2019', 'text': '@Thebault_Arthur Still working with RDW, the car regulatory authority in Netherlands. Unclear why there’s a delay.… https://t.co/9IwK1HoUs5'}, {'id': '1182823556830253056', 'date': 'Sat Oct 12 01:01:26 +0000 2019', 'text': 'Now that Tesla V10.0 with Smart Summon is out, Full Self-Driving price will increase by $1000 on Nov 1'}, {'id': '1182812717083611138', 'date': 'Sat Oct 12 00:18:22 +0000 2019', 'text': 'Get Tesla Solar plus Powerwall battery for 24/7 clean power &amp; no more blackouts! https://t.co/mDoPO17YB9'}, {'id': '1182808205425922049', 'date': 'Sat Oct 12 00:00:26 +0000 2019', 'text': '@mandeepdalip @Tesla Will be throughout Canada'}, {'id': '1182711457156190208', 'date': 'Fri Oct 11 17:35:59 +0000 2019', 'text': '@vincent13031925 It’s something special'}, {'id': '1182327145793634304', 'date': 'Thu Oct 10 16:08:52 +0000 2019', 'text': '@Gfilche @Tesla Fundamental metric of good of Tesla is how many years sooner it brings the advent of a sustainable energy future'}, {'id': '1182192515778170881', 'date': 'Thu Oct 10 07:13:54 +0000 2019', 'text': '@RationalEtienne @tesla_truth @AfMusk @28delayslater @EvaFoxU @JohnnaCrider1 @vincent13031925 @mayemusk… https://t.co/Tk7pCpZz6X'}, {'id': '1182189869344247809', 'date': 'Thu Oct 10 07:03:23 +0000 2019', 'text': '@sara_boutall @In2867Muk @parisreview I ♥️ Asterix &amp; Obelix!'}, {'id': '1182189464736526337', 'date': 'Thu Oct 10 07:01:47 +0000 2019', 'text': '@tesla_truth Exactly'}, {'id': '1182188397965656064', 'date': 'Thu Oct 10 06:57:32 +0000 2019', 'text': '@Milostoessel @Tesla @TeslaSchweiz Ok'}, {'id': '1182089703039700993', 'date': 'Thu Oct 10 00:25:22 +0000 2019', 'text': 'Also adding Tesla Solar to our Supercharger stations as fast as possible. Goal is 24/7 clean power with no blackouts.'}, {'id': '1182089226826813440', 'date': 'Thu Oct 10 00:23:28 +0000 2019', 'text': 'All Tesla Supercharger stations in regions affected by California power outages will have Tesla Powerpacks within n… https://t.co/WK81oHrGqj'}, {'id': '1182061315390492673', 'date': 'Wed Oct 09 22:32:33 +0000 2019', 'text': 'Sooo annoying!! https://t.co/QKtCC0YgDG'}, {'id': '1182059866052952066', 'date': 'Wed Oct 09 22:26:48 +0000 2019', 'text': 'https://t.co/BkKeiIzNPM'}, {'id': '1182005812975128576', 'date': 'Wed Oct 09 18:52:01 +0000 2019', 'text': '@Teslarati We’re changing Powerwall Storm Watch to Disaster Watch. Covers more bases.'}, {'id': '1181991353212817408', 'date': 'Wed Oct 09 17:54:33 +0000 2019', 'text': '@Kristennetten @annerajb @tesla_truth 🚀🚘 🧠🕳'}, {'id': '1181990505363599360', 'date': 'Wed Oct 09 17:51:11 +0000 2019', 'text': '@annerajb @tesla_truth Yes'}, {'id': '1181990398379483137', 'date': 'Wed Oct 09 17:50:45 +0000 2019', 'text': '@tesla_truth No change'}, {'id': '1181990010657984513', 'date': 'Wed Oct 09 17:49:13 +0000 2019', 'text': '@JimBridenstine @NASAMoon Important for Moonbase Alpha!'}, {'id': '1181989324797009922', 'date': 'Wed Oct 09 17:46:29 +0000 2019', 'text': '@GeorgKonjovic Working on it'}, {'id': '1181989103606255616', 'date': 'Wed Oct 09 17:45:37 +0000 2019', 'text': '@_GaryTheBot Even Gary likes it!'}, {'id': '1181988619000500224', 'date': 'Wed Oct 09 17:43:41 +0000 2019', 'text': 'It’s really quite fun!'}, {'id': '1181988380990558208', 'date': 'Wed Oct 09 17:42:44 +0000 2019', 'text': 'But we call it “caraoke”, because it’s in a 🚘'}, {'id': '1181988180326633472', 'date': 'Wed Oct 09 17:41:57 +0000 2019', 'text': 'Tesla software V10.0 comes with karaoke 🎤'}, {'id': '1181987789589508096', 'date': 'Wed Oct 09 17:40:23 +0000 2019', 'text': '@nextspaceflight Yes'}, {'id': '1181987612992524288', 'date': 'Wed Oct 09 17:39:41 +0000 2019', 'text': 'A Shortfall of Gravitas'}, {'id': '1181980074037301248', 'date': 'Wed Oct 09 17:09:44 +0000 2019', 'text': '@Thebault_Arthur @_GaryTheBot Next update'}, {'id': '1181979746550202368', 'date': 'Wed Oct 09 17:08:26 +0000 2019', 'text': '@_GaryTheBot siq burn, Gary, siq burn'}, {'id': '1181979597467873280', 'date': 'Wed Oct 09 17:07:50 +0000 2019', 'text': '🦌'}, {'id': '1181979201915604993', 'date': 'Wed Oct 09 17:06:16 +0000 2019', 'text': 'Why is there no emoji for dignity!!'}, {'id': '1181841203286790144', 'date': 'Wed Oct 09 07:57:55 +0000 2019', 'text': '@teslaownersSV Yes'}, {'id': '1181837821855711232', 'date': 'Wed Oct 09 07:44:28 +0000 2019', 'text': '@sara_boutall Door hinge'}, {'id': '1181837097444896768', 'date': 'Wed Oct 09 07:41:36 +0000 2019', 'text': 'the color orange is named after the fruit'}, {'id': '1181781720841908224', 'date': 'Wed Oct 09 04:01:33 +0000 2019', 'text': '@alex_gayer @teslaownersSV Will remove'}, {'id': '1181781181316026369', 'date': 'Wed Oct 09 03:59:24 +0000 2019', 'text': '@teslaownersSV Thanks for mentioning. Should be enabled on all loaner cars.'}, {'id': '1181781021785653248', 'date': 'Wed Oct 09 03:58:46 +0000 2019', 'text': '@Aero_JohnG @Erdayastronaut @SciGuySpace Yeah'}, {'id': '1181738919076851712', 'date': 'Wed Oct 09 01:11:28 +0000 2019', 'text': '@annerajb @Teslarati Not easily, as requires wire harness changes'}, {'id': '1181738665963208704', 'date': 'Wed Oct 09 01:10:28 +0000 2019', 'text': '@Teslarati 🦶 💨'}, {'id': '1181658171598917632', 'date': 'Tue Oct 08 19:50:36 +0000 2019', 'text': '@justpaulinelol I’m in the credits'}, {'id': '1181656686236508160', 'date': 'Tue Oct 08 19:44:42 +0000 2019', 'text': 'Worth reading “Human Compatible” by Stuart Russell (he’s great!) about future AI risks &amp; solutions https://t.co/ZCdvZrrcVf'}, {'id': '1181585837869297664', 'date': 'Tue Oct 08 15:03:11 +0000 2019', 'text': '@flcnhvy @SciGuySpace All hardware is at the Cape. Need to do static fire and reconfigure for flight. Launch probably late Nov / early Dec.'}, {'id': '1181584415362707456', 'date': 'Tue Oct 08 14:57:32 +0000 2019', 'text': '@SciGuySpace We had to reallocate some resources to speed this up &amp; received great support from Airborne, our parac… https://t.co/nuM43KoKDC'}, {'id': '1181579173388673025', 'date': 'Tue Oct 08 14:36:42 +0000 2019', 'text': '@SciGuySpace For what it’s worth, the SpaceX schedule, which I’ve just reviewed in depth, shows Falcon &amp; Dragon at… https://t.co/AloXFnfr3W'}, {'id': '1181497517759811584', 'date': 'Tue Oct 08 09:12:14 +0000 2019', 'text': '@alex_gayer 🤣'}, {'id': '1181345349148917760', 'date': 'Mon Oct 07 23:07:34 +0000 2019', 'text': '@flcnhvy @BnOrdhaug @Gizmodo Culture Class is really something'}, {'id': '1181344603716255745', 'date': 'Mon Oct 07 23:04:36 +0000 2019', 'text': '@BnOrdhaug @Gizmodo I hope so'}, {'id': '1181343438630838273', 'date': 'Mon Oct 07 22:59:58 +0000 2019', 'text': '@Gizmodo We’re doomed!'}, {'id': '1181304116498165760', 'date': 'Mon Oct 07 20:23:43 +0000 2019', 'text': '@pewdiepie Hāhā'}, {'id': '1181295442132230144', 'date': 'Mon Oct 07 19:49:15 +0000 2019', 'text': '@browserdotsys Haha true'}, {'id': '1181114635962814464', 'date': 'Mon Oct 07 07:50:47 +0000 2019', 'text': '@WellsAdams Done'}, {'id': '1181087900051492864', 'date': 'Mon Oct 07 06:04:33 +0000 2019', 'text': 'Trouble! https://t.co/qo2hR7Vcsp'}, {'id': '1181083687384838145', 'date': 'Mon Oct 07 05:47:49 +0000 2019', 'text': 'And that’s just the tip of the iceberg https://t.co/cgRKiDsdbK'}, {'id': '1181070766101549056', 'date': 'Mon Oct 07 04:56:28 +0000 2019', 'text': '@jesterpb @yukaichou @Tesla Agreed'}, {'id': '1180957431368568832', 'date': 'Sun Oct 06 21:26:07 +0000 2019', 'text': '@TameemJi @yukaichou @Tesla Good ones'}, {'id': '1180928986903265280', 'date': 'Sun Oct 06 19:33:05 +0000 2019', 'text': '@SpodieDodie Agreed, fair point'}, {'id': '1180927887970422785', 'date': 'Sun Oct 06 19:28:43 +0000 2019', 'text': '@yukaichou @Tesla Any feature requests?'}, {'id': '1180927745301176320', 'date': 'Sun Oct 06 19:28:09 +0000 2019', 'text': 'Given how quickly the world is shifting to electric vehicles, a gas/diesel vehicle bought today will probably have… https://t.co/aPRBK7hr1y'}, {'id': '1180878496224038914', 'date': 'Sun Oct 06 16:12:27 +0000 2019', 'text': '💨 &amp; 🐐 sounds too (also, of course)'}, {'id': '1180877799734665216', 'date': 'Sun Oct 06 16:09:41 +0000 2019', 'text': '@JudaMeg Will consider'}, {'id': '1180877420821237760', 'date': 'Sun Oct 06 16:08:11 +0000 2019', 'text': '@teslaownersSV Sure'}, {'id': '1180877114226008064', 'date': 'Sun Oct 06 16:06:58 +0000 2019', 'text': 'Customized horn &amp; movement sounds (coconuts being one, of course) coming to Teslas soon'}, {'id': '1180569585558228992', 'date': 'Sat Oct 05 19:44:57 +0000 2019', 'text': '@Erdayastronaut @bluemoondance74 @EvaFoxU Haha'}, {'id': '1180553336082157568', 'date': 'Sat Oct 05 18:40:23 +0000 2019', 'text': '@bluemoondance74 @EvaFoxU We will understand the next question'}, {'id': '1180553063301468160', 'date': 'Sat Oct 05 18:39:18 +0000 2019', 'text': '@justpaulinelol @EvaFoxU True'}, {'id': '1180552829498322945', 'date': 'Sat Oct 05 18:38:22 +0000 2019', 'text': '@EvaFoxU To understand the nature of consciousness'}, {'id': '1180320978435543042', 'date': 'Sat Oct 05 03:17:05 +0000 2019', 'text': '@Mikeado @Lancelot_Fox @Tommymcglynn @trevorandersen @fermatslibrary It’s true 🤣🤣'}, {'id': '1180264594364870660', 'date': 'Fri Oct 04 23:33:02 +0000 2019', 'text': '@vincent13031925 @Sofiaan @SCE Exactly'}, {'id': '1180165032690233345', 'date': 'Fri Oct 04 16:57:24 +0000 2019', 'text': '@rklau Great list of hotel hacks in this thread!'}, {'id': '1179996203574013953', 'date': 'Fri Oct 04 05:46:32 +0000 2019', 'text': '@tesla_truth 🤣🤣'}, {'id': '1179957355628253185', 'date': 'Fri Oct 04 03:12:10 +0000 2019', 'text': 'Excited about launching @NASA asteroid defense mission! #Armageddon69 https://t.co/6Hz4CKOLwU'}, {'id': '1179941711306358787', 'date': 'Fri Oct 04 02:10:00 +0000 2019', 'text': '@nichegamer Let’s go boyz!'}, {'id': '1179925839669891072', 'date': 'Fri Oct 04 01:06:56 +0000 2019', 'text': '@TeslaOwnersSK @Tesla Sorry it took so long!'}, {'id': '1179897546161848320', 'date': 'Thu Oct 03 23:14:31 +0000 2019', 'text': '@TarikCelebii Wow, cool! Summon will get steadily smoother until it’s like an expert chauffeur who knows you well.'}, {'id': '1179799146464628736', 'date': 'Thu Oct 03 16:43:30 +0000 2019', 'text': '@flcnhvy @HarryStoltz1 Welded steel'}, {'id': '1179624066375774209', 'date': 'Thu Oct 03 05:07:48 +0000 2019', 'text': '@HarryStoltz1 Probably Starlink 🛰 &amp; some fun things'}, {'id': '1179576466721259520', 'date': 'Thu Oct 03 01:58:39 +0000 2019', 'text': 'Rosetta by Vangelis is incredible https://t.co/MKqbsdVVsN'}, {'id': '1179524946478817280', 'date': 'Wed Oct 02 22:33:56 +0000 2019', 'text': '@teslaownersSV @arctechinc Not sure, will ask regulators'}, {'id': '1179524799804002304', 'date': 'Wed Oct 02 22:33:21 +0000 2019', 'text': '@AdamTForbes @annerajb @flcnhvy Enhanced should get it too'}, {'id': '1179524256792641542', 'date': 'Wed Oct 02 22:31:11 +0000 2019', 'text': '@arctechinc Aiming for this weekend!'}, {'id': '1179523171780681728', 'date': 'Wed Oct 02 22:26:53 +0000 2019', 'text': '@annerajb @flcnhvy Hopefully, a month or two, although it may be a little silly at first'}, {'id': '1179522786240319488', 'date': 'Wed Oct 02 22:25:21 +0000 2019', 'text': '@flcnhvy Not yet'}, {'id': '1179522738278395905', 'date': 'Wed Oct 02 22:25:09 +0000 2019', 'text': '@MemesOfMars First point release update expected to go out this weekend'}, {'id': '1179520622004588544', 'date': 'Wed Oct 02 22:16:45 +0000 2019', 'text': 'Over 550,000 Tesla Smart Summon uses in first few days!'}, {'id': '1179519667808768000', 'date': 'Wed Oct 02 22:12:57 +0000 2019', 'text': '@tesla_truth Thanks, Tesla team is the strongest it’s ever been &amp; doing an amazing job! Honored to work beside them.'}, {'id': '1179159128708538370', 'date': 'Tue Oct 01 22:20:18 +0000 2019', 'text': '@justpaulinelol @KeithMansfield @Erdayastronaut If 20km works, then orbit'}, {'id': '1179158176618315776', 'date': 'Tue Oct 01 22:16:31 +0000 2019', 'text': '@KeithMansfield @Erdayastronaut Great summary of the key points!'}, {'id': '1179154111616274433', 'date': 'Tue Oct 01 22:00:22 +0000 2019', 'text': '@OranMaliphant @Erdayastronaut @AngelNDevil2 @MarkChica @FutureJurvetson @bluemoondance74 @SpaceX That’s about righ… https://t.co/UuXAf1fpBh'}, {'id': '1179107730876796928', 'date': 'Tue Oct 01 18:56:04 +0000 2019', 'text': '@AngelNDevil2 @MarkChica @FutureJurvetson @bluemoondance74 @SpaceX It’s all the “secondary” structure that concerns… https://t.co/vQeVrk8J4V'}, {'id': '1179107539352313856', 'date': 'Tue Oct 01 18:55:18 +0000 2019', 'text': '@AngelNDevil2 @MarkChica @FutureJurvetson @bluemoondance74 @SpaceX Raptor cost is tracking to well under $1M for V1… https://t.co/ic6sKxZLFG'}, {'id': '1179090138644668416', 'date': 'Tue Oct 01 17:46:10 +0000 2019', 'text': '@JohnnaCrider1 @RationalEtienne @LaurenRow5 @RenataKonkoly @Kristennetten @EvaFoxU @MemesOfMars @PositivePyrami1 Stop sending me this 💩 🤣🤣'}, {'id': '1179079219801812993', 'date': 'Tue Oct 01 17:02:46 +0000 2019', 'text': '@RationalEtienne @LaurenRow5 @EvaFoxU @Kristennetten @SteveHamel16 @TheElonMasked @MemesOfMars @MichellBasler… https://t.co/O4xgr8y6kc'}, {'id': '1179078946941353984', 'date': 'Tue Oct 01 17:01:41 +0000 2019', 'text': '@Kristennetten @SS4Adrian I have that exact thought every time I get the flu'}, {'id': '1179074198540435456', 'date': 'Tue Oct 01 16:42:49 +0000 2019', 'text': '@SS4Adrian Clearly a sign of the apocalypse!'}, {'id': '1178956709731024896', 'date': 'Tue Oct 01 08:55:58 +0000 2019', 'text': '@engineers_feed Lot of ways to melt &amp; join steel. Mostly just needs to be welded consistently &amp; with precise parts.'}, {'id': '1178956134968807424', 'date': 'Tue Oct 01 08:53:41 +0000 2019', 'text': '@engineers_feed Thankfully not. FSW is very difficult to get right &amp; not needed with steel.'}, {'id': '1178938112195514368', 'date': 'Tue Oct 01 07:42:04 +0000 2019', 'text': 'Production version will be a lot more polished than this prototype, but still fun to see'}, {'id': '1178934984167448577', 'date': 'Tue Oct 01 07:29:38 +0000 2019', 'text': '@ColinPearceUK Bigger flaps/fins at rear also moves center of pressure back'}, {'id': '1178932681028980736', 'date': 'Tue Oct 01 07:20:29 +0000 2019', 'text': '@TrevorMahlmann Haha yes'}, {'id': '1178932062733996032', 'date': 'Tue Oct 01 07:18:01 +0000 2019', 'text': '@HarryStoltz1 In solving for a good Mars climate, we will learn a great deal about how to do so on Earth. It is the inverse problem.'}, {'id': '1178931253229187072', 'date': 'Tue Oct 01 07:14:48 +0000 2019', 'text': 'Inside Starship cargo bay. Header tanks mounted in tip of nosecone to offset engine weight at rear. https://t.co/EJSwqMCooA'}, {'id': '1178905464202883072', 'date': 'Tue Oct 01 05:32:20 +0000 2019', 'text': '@EvaFoxU @DisruptResearch @Tesla @karpathy Cool video!'}, {'id': '1178818555061493761', 'date': 'Mon Sep 30 23:46:59 +0000 2019', 'text': '@DisruptResearch @Tesla @karpathy Probably our most viral feature ever. Most of the Autopilot team has been working… https://t.co/Vs7de166X3'}, {'id': '1178807103273631744', 'date': 'Mon Sep 30 23:01:29 +0000 2019', 'text': '@S_Padival @Tesla @business @liamdenning @josheidelson @danahull @ZekeFaux Good question'}, {'id': '1178107012141535232', 'date': 'Sun Sep 29 00:39:34 +0000 2019', 'text': '@thejackbeyer 😀'}, {'id': '1178014342031298561', 'date': 'Sat Sep 28 18:31:20 +0000 2019', 'text': 'Rage, rage against the dying of the light https://t.co/aGRw4VowAQ'}, {'id': '1178013709546991619', 'date': 'Sat Sep 28 18:28:49 +0000 2019', 'text': '@tesla_truth @NRF_designs We’re going to use the pedestrian pink noise speaker to talk to people &amp; create customize… https://t.co/cWLXua7AnI'}, {'id': '1178012340383621120', 'date': 'Sat Sep 28 18:23:23 +0000 2019', 'text': '@tesla_truth Special light pattern (but not hazards, as they’re too shrill) seems right'}, {'id': '1178011464852004864', 'date': 'Sat Sep 28 18:19:54 +0000 2019', 'text': '@tesla_truth Makes sense'}, {'id': '1177961429703507968', 'date': 'Sat Sep 28 15:01:04 +0000 2019', 'text': '@tesla_truth @Tesla Precisely'}, {'id': '1177665592242794497', 'date': 'Fri Sep 27 19:25:31 +0000 2019', 'text': 'To make life as we know it multiplanetary'}, {'id': '1177665504699224064', 'date': 'Fri Sep 27 19:25:10 +0000 2019', 'text': 'Starship will allow us to inhabit other worlds'}, {'id': '1177662806117584896', 'date': 'Fri Sep 27 19:14:27 +0000 2019', 'text': 'Starship halves being joined https://t.co/7pdxfh8e81'}, {'id': '1177657997134221312', 'date': 'Fri Sep 27 18:55:21 +0000 2019', 'text': 'Summon is improving rapidly \nhttps://t.co/khgXDRnD2Z'}, {'id': '1177631358987849729', 'date': 'Fri Sep 27 17:09:29 +0000 2019', 'text': '@flcnhvy @Erdayastronaut @Cor_SPACE Just SX500 so far, but we use many superalloys developed by others, eg Inco 718… https://t.co/2ZRmiA1peV'}, {'id': '1177630155801419778', 'date': 'Fri Sep 27 17:04:43 +0000 2019', 'text': '@Erdayastronaut @Cor_SPACE There’s so much power going through this engine that it’s close to the limit of the know… https://t.co/7LEcDvIRi0'}, {'id': '1177628992762855426', 'date': 'Fri Sep 27 17:00:05 +0000 2019', 'text': '@Carter__99 @Erdayastronaut @askingbubble385 @Cor_SPACE Yes'}, {'id': '1177628790609997825', 'date': 'Fri Sep 27 16:59:17 +0000 2019', 'text': '@Erdayastronaut @Cor_SPACE Well, hopefully closer to 300 bar over time. However, that the chamber/nozzle jacket has… https://t.co/VWegWXxxsL'}, {'id': '1177622466727821312', 'date': 'Fri Sep 27 16:34:09 +0000 2019', 'text': '@flcnhvy @Erdayastronaut @Cor_SPACE No, we’ve been focusing on production (now on SN12). Only need 250 bar or even… https://t.co/MCIxOMAXeJ'}, {'id': '1177621843219365888', 'date': 'Fri Sep 27 16:31:41 +0000 2019', 'text': '@NCBirbhan @Erdayastronaut @Cor_SPACE Probably around 6 or 7pm'}, {'id': '1177621104686329856', 'date': 'Fri Sep 27 16:28:45 +0000 2019', 'text': '@Erdayastronaut @Cor_SPACE What blows my mind is that peak pressure in regen chamber &amp; nozzle is *far* above combus… https://t.co/QZf6knzhq9'}, {'id': '1177618467136008194', 'date': 'Fri Sep 27 16:18:16 +0000 2019', 'text': '@GaryTheBot_ Figure of speech'}, {'id': '1177618401159602176', 'date': 'Fri Sep 27 16:18:00 +0000 2019', 'text': 'Meanwhile, back in reality https://t.co/NrHHB9G81L'}, {'id': '1177618032719319040', 'date': 'Fri Sep 27 16:16:32 +0000 2019', 'text': '@DLoesch Media report was false'}, {'id': '1177387141116002304', 'date': 'Fri Sep 27 00:59:03 +0000 2019', 'text': '@Erdayastronaut @Cor_SPACE Raptor uses milled copper channels with an inconel jacket all the way down'}, {'id': '1177314408604680192', 'date': 'Thu Sep 26 20:10:03 +0000 2019', 'text': 'Three Raptors on a Starship https://t.co/UrRiD62EVk'}, {'id': '1177259561117372417', 'date': 'Thu Sep 26 16:32:06 +0000 2019', 'text': '@EvaFoxU @BrendanCogbill7 Always'}, {'id': '1177241843093753862', 'date': 'Thu Sep 26 15:21:42 +0000 2019', 'text': '@SpacePadreIsle @SpaceX Wow, it’s been 5 years since we started at Boca!'}, {'id': '1177237783489863681', 'date': 'Thu Sep 26 15:05:34 +0000 2019', 'text': '@BrendanCogbill7 About a week. Will include slight improvements, based on initial user feedback.'}, {'id': '1177236839406559232', 'date': 'Thu Sep 26 15:01:49 +0000 2019', 'text': '@justpaulinelol We look for rare corner case issues with a limited release US build, then expand internationally as… https://t.co/Lqw9n2ZHvR'}, {'id': '1177235963405193216', 'date': 'Thu Sep 26 14:58:20 +0000 2019', 'text': '@GaryTheBot_ Gary, you rock!'}, {'id': '1177235772857970688', 'date': 'Thu Sep 26 14:57:34 +0000 2019', 'text': 'Tesla V10.0 just started rolling out to all US Tesla owners with FSD option &amp; “advanced download” selected in vehicle software settings'}, {'id': '1177101142980825093', 'date': 'Thu Sep 26 06:02:36 +0000 2019', 'text': '@moyo5150 More like https://t.co/f8BUlz0Xdo'}, {'id': '1177100271937212416', 'date': 'Thu Sep 26 05:59:08 +0000 2019', 'text': 'Anonymous bot swarms deserve a closer examination. If they’re evolving rapidly, something’s up.'}, {'id': '1177099375287271424', 'date': 'Thu Sep 26 05:55:35 +0000 2019', 'text': 'If advanced AI (beyond basic bots) hasn’t been applied to manipulate social media, it won’t be long before it is'}, {'id': '1177096273628778496', 'date': 'Thu Sep 26 05:43:15 +0000 2019', 'text': '@Wally_Wests Exactly'}, {'id': '1177090550916239360', 'date': 'Thu Sep 26 05:20:31 +0000 2019', 'text': '@Kristennetten @annerajb @Jennerator211 @Erdayastronaut That would be a nice touch'}, {'id': '1177085193007456256', 'date': 'Thu Sep 26 04:59:13 +0000 2019', 'text': '@flcnhvy @28delayslater @DJSnM This is quite a complex beast, but hopefully within a month or so'}, {'id': '1177084577128448000', 'date': 'Thu Sep 26 04:56:47 +0000 2019', 'text': '@28delayslater @DJSnM Haha awesome'}, {'id': '1177074975020220416', 'date': 'Thu Sep 26 04:18:37 +0000 2019', 'text': '@MarkChica @Kristennetten @TeslaGong @teslaownersSV @Teslatunity @Tesla We could probably do that'}, {'id': '1177070801121468418', 'date': 'Thu Sep 26 04:02:02 +0000 2019', 'text': '@Erdayastronaut 50m'}, {'id': '1177069611142901762', 'date': 'Thu Sep 26 03:57:18 +0000 2019', 'text': '@TeslaGong @teslaownersSV @Teslatunity @Tesla Not sure we could go quite that tight, but we can for sure allow much… https://t.co/wolYBBZMS7'}, {'id': '1177067257106747392', 'date': 'Thu Sep 26 03:47:57 +0000 2019', 'text': '@annerajb @Jennerator211 @Erdayastronaut With rare exceptions, composites would make Starship heavier. They don’t s… https://t.co/OeeigvDiI8'}, {'id': '1177066483375058944', 'date': 'Thu Sep 26 03:44:53 +0000 2019', 'text': '@Jennerator211 @Erdayastronaut Mk1 ship is around 200 tons dry &amp; 1400 tons wet, but aiming for 120 by Mk4 or Mk5. T… https://t.co/yTUpGx0Ppa'}, {'id': '1177064952429010945', 'date': 'Thu Sep 26 03:38:48 +0000 2019', 'text': '@teslaownersSV @Teslatunity @Tesla Absolutely'}, {'id': '1177062354577756160', 'date': 'Thu Sep 26 03:28:28 +0000 2019', 'text': '@Teslatunity @Tesla You’re welcome! Feature requests always appreciated.'}, {'id': '1177051800769138690', 'date': 'Thu Sep 26 02:46:32 +0000 2019', 'text': '@Erdayastronaut @kimitalvitie Six. Two windward, one under each fin &amp; two leeward. Provides redundancy for landing on unimproved surfaces.'}, {'id': '1177050841041125376', 'date': 'Thu Sep 26 02:42:43 +0000 2019', 'text': 'On the transporter https://t.co/CGNj1MlKAU'}, {'id': '1177049277375250434', 'date': 'Thu Sep 26 02:36:30 +0000 2019', 'text': 'Lifting lower section of Starship https://t.co/bfW17i469a'}, {'id': '1176936806928879616', 'date': 'Wed Sep 25 19:09:35 +0000 2019', 'text': '@justpaulinelol @EvaFoxU @katlinegrey https://t.co/IzVngWWY7G'}, {'id': '1176930839411642368', 'date': 'Wed Sep 25 18:45:53 +0000 2019', 'text': '@EvaFoxU @katlinegrey правда'}, {'id': '1176928113285971968', 'date': 'Wed Sep 25 18:35:03 +0000 2019', 'text': '@katlinegrey язык хорошо подвешен'}, {'id': '1176737065624453125', 'date': 'Wed Sep 25 05:55:53 +0000 2019', 'text': '@levie Exactly'}, {'id': '1176596320775094273', 'date': 'Tue Sep 24 20:36:37 +0000 2019', 'text': '@tannerfox Impressive work by Chad &amp; Steve!'}, {'id': '1176589331567452161', 'date': 'Tue Sep 24 20:08:51 +0000 2019', 'text': '@SamNazarius @HarryStoltz1 @justpaulinelol @Teslarati Almost entirely C++, but we did have an (advanced for the tim… https://t.co/0SbJqgtWsG'}, {'id': '1176588054670323713', 'date': 'Tue Sep 24 20:03:46 +0000 2019', 'text': '@HarryStoltz1 @justpaulinelol @Teslarati Turbo Pascal'}, {'id': '1176587698401947650', 'date': 'Tue Sep 24 20:02:21 +0000 2019', 'text': '@justpaulinelol @Teslarati Mostly C/C++, Python &amp; SQL'}, {'id': '1176586684751908872', 'date': 'Tue Sep 24 19:58:20 +0000 2019', 'text': '@flcnhvy @Erdayastronaut @Tesla It would be pretty embarrassing to use a non-Tesla battery!'}, {'id': '1176585904091275264', 'date': 'Tue Sep 24 19:55:14 +0000 2019', 'text': '@Teslarati This includes original engineering of new factory processes &amp; cars. Great engineers will only join if or… https://t.co/zCAndlhS72'}]

date3 = []
for i in range(tweet.shape[0]):
    a = parse(d.date[i])
    date2 = a.strftime("%Y-%m-%d")
    date3.append(date2)

AttributeError: 'list' object has no attribute 'date'

In [181]:
date3 = pd.DataFrame(date3)

In [192]:
d2 = pd.concat([d, date3], axis = 1)

In [196]:
d2.date = d2.iloc[:,[-1]]

In [201]:
d2 = d2[['id', 'date', 'text']]

id        date  \
0  1183223595008266241  2019-10-13   
1  1183222226130046980  2019-10-13   
2  1183221390016536581  2019-10-13   
3  1183220879951462400  2019-10-13   
4  1183201528036786178  2019-10-13   

                                                text  
0  @jay11w Recommended for sure. Both movies are ...  
1  First will rewatch Godfather Part II, then Spa...  
2                    Underappreciated 🎥, will watch!  
3                                 Is Space Jam good?  
4     @Nobody_Really7 @anonyx10 @mayemusk So it goes

In [229]:
#fetching the data
d = getStuffFromTwitter
#d = [{'id': '1183223595008266241', 'date': 'Sun Oct 13 03:31:02 +0000 2019', 'text': '@jay11w Recommended for sure. Both movies are incredible.'}, {'id': '1183222226130046980', 'date': 'Sun Oct 13 03:25:36 +0000 2019', 'text': 'First will rewatch Godfather Part II, then Space Jam'}, {'id': '1183221390016536581', 'date': 'Sun Oct 13 03:22:17 +0000 2019', 'text': 'Underappreciated , will watch!'}, {'id': '1183220879951462400', 'date': 'Sun Oct 13 03:20:15 +0000 2019', 'text': 'Is Space Jam good?'}, {'id': '1183201528036786178', 'date': 'Sun Oct 13 02:03:21 +0000 2019', 'text': '@NobodyReally7 @anonyx10 @mayemusk So it goes'}, {'id': '1183198657136324610', 'date': 'Sun Oct 13 01:51:57 +0000 2019', 'text': '@anonyx10 @mayemusk Sure looks that way'}, {'id': '1183152738382667777', 'date': 'Sat Oct 12 22:49:29 +0000 2019', 'text': '@owlmaster08 @Tesla Nice '}, {'id': '1183150916473516032', 'date': 'Sat Oct 12 22:42:15 +0000 2019', 'text': '@Erdayastronaut Solar'}, {'id': '1183150883162349569', 'date': 'Sat Oct 12 22:42:07 +0000 2019', 'text': '@Erdayastronaut Depends on total system efficiency &amp; how long the propellant plant can run to refill Starship, so 1 to 10MW as a rough guess'}, {'id': '1183123445787185152', 'date': 'Sat Oct 12 20:53:05 +0000 2019', 'text': '@bonnienorman Whoa, that does look good!'}, {'id': '1183119389211549696', 'date': 'Sat Oct 12 20:36:58 +0000 2019', 'text': '@Macwars Fair point. Will hold on EU price change until Tesla gets clarity from regulators about Smart Summon feature release date.'}, {'id': '1183118245458432001', 'date': 'Sat Oct 12 20:32:25 +0000 2019', 'text': 'French fries  are the devil  https://t.co/AR3h57RPIA'}, {'id': '1183114722989621248', 'date': 'Sat Oct 12 20:18:25 +0000 2019', 'text': '@teslatruth @BarkMSmeagol @vincent13031925 @jchybow @JohnnaCrider1 @28delayslater @flcnhvy @AfMusk . https://t.co/vplI7NQDb6'}, {'id': '1183106801769697280', 'date': 'Sat Oct 12 19:46:57 +0000 2019', 'text': '@teslatruth @BarkMSmeagol @vincent13031925 @jchybow @JohnnaCrider1 @28delayslater @flcnhvy @AfMusk Super messed up situation!'}, {'id': '1183082764515414016', 'date': 'Sat Oct 12 18:11:26 +0000 2019', 'text': '@cleantechnica Several finesse improvements coming to Smart Summon in coming weeks. Will be smooth as silk.'}, {'id': '1183081171644628992', 'date': 'Sat Oct 12 18:05:06 +0000 2019', 'text': '@wk057 @Tesla Should be much better at this point'}, {'id': '1183033425529147392', 'date': 'Sat Oct 12 14:55:23 +0000 2019', 'text': 'Nevada has an additional $3000 rebate for battery storage with solar https://t.co/bRibVqomzy'}, {'id': '1182828744748044288', 'date': 'Sat Oct 12 01:22:03 +0000 2019', 'text': 'Price for fully installed solar plus battery has been reduced 10% to help those affected by blackouts'}, {'id': '1182828388852977665', 'date': 'Sat Oct 12 01:20:38 +0000 2019', 'text': '@brockm Yes, but probably a year or two away. Lot of fish to fry '}, {'id': '1182828056416641025', 'date': 'Sat Oct 12 01:19:19 +0000 2019', 'text': '@brucemittleman Tesla cannot continue to lose money. To achieve our goal of environmental sustainability, Tesla mu https://t.co/ArWGP4U79u'}, {'id': '1182827574361124864', 'date': 'Sat Oct 12 01:17:24 +0000 2019', 'text': 'Allows homeowner potential to go completely off grid'}, {'id': '1182827425551380482', 'date': 'Sat Oct 12 01:16:48 +0000 2019', 'text': 'V important: pure solar subscription is guaranteed to be a net money-maker for the homeowner or Tesla will take the https://t.co/m3Z9eRHGzV'}, {'id': '1182826581070245888', 'date': 'Sat Oct 12 01:13:27 +0000 2019', 'text': '@cutza7 When the car is FSD without supervision, ie robotaxi, youll be able to earn far more than monthly lease/lo https://t.co/0wyhUAyfnu'}, {'id': '1182825609686241280', 'date': 'Sat Oct 12 01:09:35 +0000 2019', 'text': '@teslaaddicted @sanjeevr11 @vincent13031925 Putting panels on east or west facing roofs works well too. Energy lo https://t.co/wG45nxMkNw'}, {'id': '1182824968200937472', 'date': 'Sat Oct 12 01:07:02 +0000 2019', 'text': '@wilsonlam '}, {'id': '1182824823275155456', 'date': 'Sat Oct 12 01:06:28 +0000 2019', 'text': '@annerajb @ThebaultArthur Working with engineering team to figure out best way to do upgrade without crushing serv https://t.co/vENrP11kKA'}, {'id': '1182824078756872192', 'date': 'Sat Oct 12 01:03:30 +0000 2019', 'text': '@ThebaultArthur Still working with RDW, the car regulatory authority in Netherlands. Unclear why theres a delay. https://t.co/9IwK1HoUs5'}, {'id': '1182823556830253056', 'date': 'Sat Oct 12 01:01:26 +0000 2019', 'text': 'Now that Tesla V10.0 with Smart Summon is out, Full Self-Driving price will increase by $1000 on Nov 1'}, {'id': '1182812717083611138', 'date': 'Sat Oct 12 00:18:22 +0000 2019', 'text': 'Get Tesla Solar plus Powerwall battery for 24/7 clean power &amp; no more blackouts! https://t.co/mDoPO17YB9'}, {'id': '1182808205425922049', 'date': 'Sat Oct 12 00:00:26 +0000 2019', 'text': '@mandeepdalip @Tesla Will be throughout Canada'}, {'id': '1182711457156190208', 'date': 'Fri Oct 11 17:35:59 +0000 2019', 'text': '@vincent13031925 Its something special'}, {'id': '1182327145793634304', 'date': 'Thu Oct 10 16:08:52 +0000 2019', 'text': '@Gfilche @Tesla Fundamental metric of good of Tesla is how many years sooner it brings the advent of a sustainable energy future'}, {'id': '1182192515778170881', 'date': 'Thu Oct 10 07:13:54 +0000 2019', 'text': '@RationalEtienne @teslatruth @AfMusk @28delayslater @EvaFoxU @JohnnaCrider1 @vincent13031925 @mayemusk https://t.co/Tk7pCpZz6X'}, {'id': '1182189869344247809', 'date': 'Thu Oct 10 07:03:23 +0000 2019', 'text': '@saraboutall @In2867Muk @parisreview I  Asterix &amp; Obelix!'}, {'id': '1182189464736526337', 'date': 'Thu Oct 10 07:01:47 +0000 2019', 'text': '@teslatruth Exactly'}, {'id': '1182188397965656064', 'date': 'Thu Oct 10 06:57:32 +0000 2019', 'text': '@Milostoessel @Tesla @TeslaSchweiz Ok'}, {'id': '1182089703039700993', 'date': 'Thu Oct 10 00:25:22 +0000 2019', 'text': 'Also adding Tesla Solar to our Supercharger stations as fast as possible. Goal is 24/7 clean power with no blackouts.'}, {'id': '1182089226826813440', 'date': 'Thu Oct 10 00:23:28 +0000 2019', 'text': 'All Tesla Supercharger stations in regions affected by California power outages will have Tesla Powerpacks within n https://t.co/WK81oHrGqj'}, {'id': '1182061315390492673', 'date': 'Wed Oct 09 22:32:33 +0000 2019', 'text': 'Sooo annoying!! https://t.co/QKtCC0YgDG'}, {'id': '1182059866052952066', 'date': 'Wed Oct 09 22:26:48 +0000 2019', 'text': 'https://t.co/BkKeiIzNPM'}, {'id': '1182005812975128576', 'date': 'Wed Oct 09 18:52:01 +0000 2019', 'text': '@Teslarati Were changing Powerwall Storm Watch to Disaster Watch. Covers more bases.'}, {'id': '1181991353212817408', 'date': 'Wed Oct 09 17:54:33 +0000 2019', 'text': '@Kristennetten @annerajb @teslatruth  '}, {'id': '1181990505363599360', 'date': 'Wed Oct 09 17:51:11 +0000 2019', 'text': '@annerajb @teslatruth Yes'}, {'id': '1181990398379483137', 'date': 'Wed Oct 09 17:50:45 +0000 2019', 'text': '@teslatruth No change'}, {'id': '1181990010657984513', 'date': 'Wed Oct 09 17:49:13 +0000 2019', 'text': '@JimBridenstine @NASAMoon Important for Moonbase Alpha!'}, {'id': '1181989324797009922', 'date': 'Wed Oct 09 17:46:29 +0000 2019', 'text': '@GeorgKonjovic Working on it'}, {'id': '1181989103606255616', 'date': 'Wed Oct 09 17:45:37 +0000 2019', 'text': '@GaryTheBot Even Gary likes it!'}, {'id': '1181988619000500224', 'date': 'Wed Oct 09 17:43:41 +0000 2019', 'text': 'Its really quite fun!'}, {'id': '1181988380990558208', 'date': 'Wed Oct 09 17:42:44 +0000 2019', 'text': 'But we call it caraoke, because its in a '}, {'id': '1181988180326633472', 'date': 'Wed Oct 09 17:41:57 +0000 2019', 'text': 'Tesla software V10.0 comes with karaoke '}, {'id': '1181987789589508096', 'date': 'Wed Oct 09 17:40:23 +0000 2019', 'text': '@nextspaceflight Yes'}, {'id': '1181987612992524288', 'date': 'Wed Oct 09 17:39:41 +0000 2019', 'text': 'A Shortfall of Gravitas'}, {'id': '1181980074037301248', 'date': 'Wed Oct 09 17:09:44 +0000 2019', 'text': '@ThebaultArthur @GaryTheBot Next update'}, {'id': '1181979746550202368', 'date': 'Wed Oct 09 17:08:26 +0000 2019', 'text': '@GaryTheBot siq burn, Gary, siq burn'}, {'id': '1181979597467873280', 'date': 'Wed Oct 09 17:07:50 +0000 2019', 'text': ''}, {'id': '1181979201915604993', 'date': 'Wed Oct 09 17:06:16 +0000 2019', 'text': 'Why is there no emoji for dignity!!'}, {'id': '1181841203286790144', 'date': 'Wed Oct 09 07:57:55 +0000 2019', 'text': '@teslaownersSV Yes'}, {'id': '1181837821855711232', 'date': 'Wed Oct 09 07:44:28 +0000 2019', 'text': '@saraboutall Door hinge'}, {'id': '1181837097444896768', 'date': 'Wed Oct 09 07:41:36 +0000 2019', 'text': 'the color orange is named after the fruit'}, {'id': '1181781720841908224', 'date': 'Wed Oct 09 04:01:33 +0000 2019', 'text': '@alexgayer @teslaownersSV Will remove'}, {'id': '1181781181316026369', 'date': 'Wed Oct 09 03:59:24 +0000 2019', 'text': '@teslaownersSV Thanks for mentioning. Should be enabled on all loaner cars.'}, {'id': '1181781021785653248', 'date': 'Wed Oct 09 03:58:46 +0000 2019', 'text': '@AeroJohnG @Erdayastronaut @SciGuySpace Yeah'}, {'id': '1181738919076851712', 'date': 'Wed Oct 09 01:11:28 +0000 2019', 'text': '@annerajb @Teslarati Not easily, as requires wire harness changes'}, {'id': '1181738665963208704', 'date': 'Wed Oct 09 01:10:28 +0000 2019', 'text': '@Teslarati  '}, {'id': '1181658171598917632', 'date': 'Tue Oct 08 19:50:36 +0000 2019', 'text': '@justpaulinelol Im in the credits'}, {'id': '1181656686236508160', 'date': 'Tue Oct 08 19:44:42 +0000 2019', 'text': 'Worth reading Human Compatible by Stuart Russell (hes great!) about future AI risks &amp; solutions https://t.co/ZCdvZrrcVf'}, {'id': '1181585837869297664', 'date': 'Tue Oct 08 15:03:11 +0000 2019', 'text': '@flcnhvy @SciGuySpace All hardware is at the Cape. Need to do static fire and reconfigure for flight. Launch probably late Nov / early Dec.'}, {'id': '1181584415362707456', 'date': 'Tue Oct 08 14:57:32 +0000 2019', 'text': '@SciGuySpace We had to reallocate some resources to speed this up &amp; received great support from Airborne, our parac https://t.co/nuM43KoKDC'}, {'id': '1181579173388673025', 'date': 'Tue Oct 08 14:36:42 +0000 2019', 'text': '@SciGuySpace For what its worth, the SpaceX schedule, which Ive just reviewed in depth, shows Falcon &amp; Dragon at https://t.co/AloXFnfr3W'}, {'id': '1181497517759811584', 'date': 'Tue Oct 08 09:12:14 +0000 2019', 'text': '@alexgayer '}, {'id': '1181345349148917760', 'date': 'Mon Oct 07 23:07:34 +0000 2019', 'text': '@flcnhvy @BnOrdhaug @Gizmodo Culture Class is really something'}, {'id': '1181344603716255745', 'date': 'Mon Oct 07 23:04:36 +0000 2019', 'text': '@BnOrdhaug @Gizmodo I hope so'}, {'id': '1181343438630838273', 'date': 'Mon Oct 07 22:59:58 +0000 2019', 'text': '@Gizmodo Were doomed!'}, {'id': '1181304116498165760', 'date': 'Mon Oct 07 20:23:43 +0000 2019', 'text': '@pewdiepie Hh'}, {'id': '1181295442132230144', 'date': 'Mon Oct 07 19:49:15 +0000 2019', 'text': '@browserdotsys Haha true'}, {'id': '1181114635962814464', 'date': 'Mon Oct 07 07:50:47 +0000 2019', 'text': '@WellsAdams Done'}, {'id': '1181087900051492864', 'date': 'Mon Oct 07 06:04:33 +0000 2019', 'text': 'Trouble! https://t.co/qo2hR7Vcsp'}, {'id': '1181083687384838145', 'date': 'Mon Oct 07 05:47:49 +0000 2019', 'text': 'And thats just the tip of the iceberg https://t.co/cgRKiDsdbK'}, {'id': '1181070766101549056', 'date': 'Mon Oct 07 04:56:28 +0000 2019', 'text': '@jesterpb @yukaichou @Tesla Agreed'}, {'id': '1180957431368568832', 'date': 'Sun Oct 06 21:26:07 +0000 2019', 'text': '@TameemJi @yukaichou @Tesla Good ones'}, {'id': '1180928986903265280', 'date': 'Sun Oct 06 19:33:05 +0000 2019', 'text': '@SpodieDodie Agreed, fair point'}, {'id': '1180927887970422785', 'date': 'Sun Oct 06 19:28:43 +0000 2019', 'text': '@yukaichou @Tesla Any feature requests?'}, {'id': '1180927745301176320', 'date': 'Sun Oct 06 19:28:09 +0000 2019', 'text': 'Given how quickly the world is shifting to electric vehicles, a gas/diesel vehicle bought today will probably have https://t.co/aPRBK7hr1y'}, {'id': '1180878496224038914', 'date': 'Sun Oct 06 16:12:27 +0000 2019', 'text': ' &amp;  sounds too (also, of course)'}, {'id': '1180877799734665216', 'date': 'Sun Oct 06 16:09:41 +0000 2019', 'text': '@JudaMeg Will consider'}, {'id': '1180877420821237760', 'date': 'Sun Oct 06 16:08:11 +0000 2019', 'text': '@teslaownersSV Sure'}, {'id': '1180877114226008064', 'date': 'Sun Oct 06 16:06:58 +0000 2019', 'text': 'Customized horn &amp; movement sounds (coconuts being one, of course) coming to Teslas soon'}, {'id': '1180569585558228992', 'date': 'Sat Oct 05 19:44:57 +0000 2019', 'text': '@Erdayastronaut @bluemoondance74 @EvaFoxU Haha'}, {'id': '1180553336082157568', 'date': 'Sat Oct 05 18:40:23 +0000 2019', 'text': '@bluemoondance74 @EvaFoxU We will understand the next question'}, {'id': '1180553063301468160', 'date': 'Sat Oct 05 18:39:18 +0000 2019', 'text': '@justpaulinelol @EvaFoxU True'}, {'id': '1180552829498322945', 'date': 'Sat Oct 05 18:38:22 +0000 2019', 'text': '@EvaFoxU To understand the nature of consciousness'}, {'id': '1180320978435543042', 'date': 'Sat Oct 05 03:17:05 +0000 2019', 'text': '@Mikeado @LancelotFox @Tommymcglynn @trevorandersen @fermatslibrary Its true '}, {'id': '1180264594364870660', 'date': 'Fri Oct 04 23:33:02 +0000 2019', 'text': '@vincent13031925 @Sofiaan @SCE Exactly'}, {'id': '1180165032690233345', 'date': 'Fri Oct 04 16:57:24 +0000 2019', 'text': '@rklau Great list of hotel hacks in this thread!'}, {'id': '1179996203574013953', 'date': 'Fri Oct 04 05:46:32 +0000 2019', 'text': '@teslatruth '}, {'id': '1179957355628253185', 'date': 'Fri Oct 04 03:12:10 +0000 2019', 'text': 'Excited about launching @NASA asteroid defense mission! #Armageddon69 https://t.co/6Hz4CKOLwU'}, {'id': '1179941711306358787', 'date': 'Fri Oct 04 02:10:00 +0000 2019', 'text': '@nichegamer Lets go boyz!'}, {'id': '1179925839669891072', 'date': 'Fri Oct 04 01:06:56 +0000 2019', 'text': '@TeslaOwnersSK @Tesla Sorry it took so long!'}, {'id': '1179897546161848320', 'date': 'Thu Oct 03 23:14:31 +0000 2019', 'text': '@TarikCelebii Wow, cool! Summon will get steadily smoother until its like an expert chauffeur who knows you well.'}, {'id': '1179799146464628736', 'date': 'Thu Oct 03 16:43:30 +0000 2019', 'text': '@flcnhvy @HarryStoltz1 Welded steel'}, {'id': '1179624066375774209', 'date': 'Thu Oct 03 05:07:48 +0000 2019', 'text': '@HarryStoltz1 Probably Starlink  &amp; some fun things'}, {'id': '1179576466721259520', 'date': 'Thu Oct 03 01:58:39 +0000 2019', 'text': 'Rosetta by Vangelis is incredible https://t.co/MKqbsdVVsN'}, {'id': '1179524946478817280', 'date': 'Wed Oct 02 22:33:56 +0000 2019', 'text': '@teslaownersSV @arctechinc Not sure, will ask regulators'}, {'id': '1179524799804002304', 'date': 'Wed Oct 02 22:33:21 +0000 2019', 'text': '@AdamTForbes @annerajb @flcnhvy Enhanced should get it too'}, {'id': '1179524256792641542', 'date': 'Wed Oct 02 22:31:11 +0000 2019', 'text': '@arctechinc Aiming for this weekend!'}, {'id': '1179523171780681728', 'date': 'Wed Oct 02 22:26:53 +0000 2019', 'text': '@annerajb @flcnhvy Hopefully, a month or two, although it may be a little silly at first'}, {'id': '1179522786240319488', 'date': 'Wed Oct 02 22:25:21 +0000 2019', 'text': '@flcnhvy Not yet'}, {'id': '1179522738278395905', 'date': 'Wed Oct 02 22:25:09 +0000 2019', 'text': '@MemesOfMars First point release update expected to go out this weekend'}, {'id': '1179520622004588544', 'date': 'Wed Oct 02 22:16:45 +0000 2019', 'text': 'Over 550,000 Tesla Smart Summon uses in first few days!'}, {'id': '1179519667808768000', 'date': 'Wed Oct 02 22:12:57 +0000 2019', 'text': '@teslatruth Thanks, Tesla team is the strongest its ever been &amp; doing an amazing job! Honored to work beside them.'}, {'id': '1179159128708538370', 'date': 'Tue Oct 01 22:20:18 +0000 2019', 'text': '@justpaulinelol @KeithMansfield @Erdayastronaut If 20km works, then orbit'}, {'id': '1179158176618315776', 'date': 'Tue Oct 01 22:16:31 +0000 2019', 'text': '@KeithMansfield @Erdayastronaut Great summary of the key points!'}, {'id': '1179154111616274433', 'date': 'Tue Oct 01 22:00:22 +0000 2019', 'text': '@OranMaliphant @Erdayastronaut @AngelNDevil2 @MarkChica @FutureJurvetson @bluemoondance74 @SpaceX Thats about righ https://t.co/UuXAf1fpBh'}, {'id': '1179107730876796928', 'date': 'Tue Oct 01 18:56:04 +0000 2019', 'text': '@AngelNDevil2 @MarkChica @FutureJurvetson @bluemoondance74 @SpaceX Its all the secondary structure that concerns https://t.co/vQeVrk8J4V'}, {'id': '1179107539352313856', 'date': 'Tue Oct 01 18:55:18 +0000 2019', 'text': '@AngelNDevil2 @MarkChica @FutureJurvetson @bluemoondance74 @SpaceX Raptor cost is tracking to well under $1M for V1 https://t.co/ic6sKxZLFG'}, {'id': '1179090138644668416', 'date': 'Tue Oct 01 17:46:10 +0000 2019', 'text': '@JohnnaCrider1 @RationalEtienne @LaurenRow5 @RenataKonkoly @Kristennetten @EvaFoxU @MemesOfMars @PositivePyrami1 Stop sending me this  '}, {'id': '1179079219801812993', 'date': 'Tue Oct 01 17:02:46 +0000 2019', 'text': '@RationalEtienne @LaurenRow5 @EvaFoxU @Kristennetten @SteveHamel16 @TheElonMasked @MemesOfMars @MichellBasler https://t.co/O4xgr8y6kc'}, {'id': '1179078946941353984', 'date': 'Tue Oct 01 17:01:41 +0000 2019', 'text': '@Kristennetten @SS4Adrian I have that exact thought every time I get the flu'}, {'id': '1179074198540435456', 'date': 'Tue Oct 01 16:42:49 +0000 2019', 'text': '@SS4Adrian Clearly a sign of the apocalypse!'}, {'id': '1178956709731024896', 'date': 'Tue Oct 01 08:55:58 +0000 2019', 'text': '@engineersfeed Lot of ways to melt &amp; join steel. Mostly just needs to be welded consistently &amp; with precise parts.'}, {'id': '1178956134968807424', 'date': 'Tue Oct 01 08:53:41 +0000 2019', 'text': '@engineersfeed Thankfully not. FSW is very difficult to get right &amp; not needed with steel.'}, {'id': '1178938112195514368', 'date': 'Tue Oct 01 07:42:04 +0000 2019', 'text': 'Production version will be a lot more polished than this prototype, but still fun to see'}, {'id': '1178934984167448577', 'date': 'Tue Oct 01 07:29:38 +0000 2019', 'text': '@ColinPearceUK Bigger flaps/fins at rear also moves center of pressure back'}, {'id': '1178932681028980736', 'date': 'Tue Oct 01 07:20:29 +0000 2019', 'text': '@TrevorMahlmann Haha yes'}, {'id': '1178932062733996032', 'date': 'Tue Oct 01 07:18:01 +0000 2019', 'text': '@HarryStoltz1 In solving for a good Mars climate, we will learn a great deal about how to do so on Earth. It is the inverse problem.'}, {'id': '1178931253229187072', 'date': 'Tue Oct 01 07:14:48 +0000 2019', 'text': 'Inside Starship cargo bay. Header tanks mounted in tip of nosecone to offset engine weight at rear. https://t.co/EJSwqMCooA'}, {'id': '1178905464202883072', 'date': 'Tue Oct 01 05:32:20 +0000 2019', 'text': '@EvaFoxU @DisruptResearch @Tesla @karpathy Cool video!'}, {'id': '1178818555061493761', 'date': 'Mon Sep 30 23:46:59 +0000 2019', 'text': '@DisruptResearch @Tesla @karpathy Probably our most viral feature ever. Most of the Autopilot team has been working https://t.co/Vs7de166X3'}, {'id': '1178807103273631744', 'date': 'Mon Sep 30 23:01:29 +0000 2019', 'text': '@SPadival @Tesla @business @liamdenning @josheidelson @danahull @ZekeFaux Good question'}, {'id': '1178107012141535232', 'date': 'Sun Sep 29 00:39:34 +0000 2019', 'text': '@thejackbeyer '}, {'id': '1178014342031298561', 'date': 'Sat Sep 28 18:31:20 +0000 2019', 'text': 'Rage, rage against the dying of the light https://t.co/aGRw4VowAQ'}, {'id': '1178013709546991619', 'date': 'Sat Sep 28 18:28:49 +0000 2019', 'text': '@teslatruth @NRFdesigns Were going to use the pedestrian pink noise speaker to talk to people &amp; create customize https://t.co/cWLXua7AnI'}, {'id': '1178012340383621120', 'date': 'Sat Sep 28 18:23:23 +0000 2019', 'text': '@teslatruth Special light pattern (but not hazards, as theyre too shrill) seems right'}, {'id': '1178011464852004864', 'date': 'Sat Sep 28 18:19:54 +0000 2019', 'text': '@teslatruth Makes sense'}, {'id': '1177961429703507968', 'date': 'Sat Sep 28 15:01:04 +0000 2019', 'text': '@teslatruth @Tesla Precisely'}, {'id': '1177665592242794497', 'date': 'Fri Sep 27 19:25:31 +0000 2019', 'text': 'To make life as we know it multiplanetary'}, {'id': '1177665504699224064', 'date': 'Fri Sep 27 19:25:10 +0000 2019', 'text': 'Starship will allow us to inhabit other worlds'}, {'id': '1177662806117584896', 'date': 'Fri Sep 27 19:14:27 +0000 2019', 'text': 'Starship halves being joined https://t.co/7pdxfh8e81'}, {'id': '1177657997134221312', 'date': 'Fri Sep 27 18:55:21 +0000 2019', 'text': 'Summon is improving rapidly https://t.co/khgXDRnD2Z'}, {'id': '1177631358987849729', 'date': 'Fri Sep 27 17:09:29 +0000 2019', 'text': '@flcnhvy @Erdayastronaut @CorSPACE Just SX500 so far, but we use many superalloys developed by others, eg Inco 718 https://t.co/2ZRmiA1peV'}, {'id': '1177630155801419778', 'date': 'Fri Sep 27 17:04:43 +0000 2019', 'text': '@Erdayastronaut @CorSPACE Theres so much power going through this engine that its close to the limit of the know https://t.co/7LEcDvIRi0'}, {'id': '1177628992762855426', 'date': 'Fri Sep 27 17:00:05 +0000 2019', 'text': '@Carter99 @Erdayastronaut @askingbubble385 @CorSPACE Yes'}, {'id': '1177628790609997825', 'date': 'Fri Sep 27 16:59:17 +0000 2019', 'text': '@Erdayastronaut @CorSPACE Well, hopefully closer to 300 bar over time. However, that the chamber/nozzle jacket has https://t.co/VWegWXxxsL'}, {'id': '1177622466727821312', 'date': 'Fri Sep 27 16:34:09 +0000 2019', 'text': '@flcnhvy @Erdayastronaut @CorSPACE No, weve been focusing on production (now on SN12). Only need 250 bar or even https://t.co/MCIxOMAXeJ'}, {'id': '1177621843219365888', 'date': 'Fri Sep 27 16:31:41 +0000 2019', 'text': '@NCBirbhan @Erdayastronaut @CorSPACE Probably around 6 or 7pm'}, {'id': '1177621104686329856', 'date': 'Fri Sep 27 16:28:45 +0000 2019', 'text': '@Erdayastronaut @CorSPACE What blows my mind is that peak pressure in regen chamber &amp; nozzle is *far* above combus https://t.co/QZf6knzhq9'}, {'id': '1177618467136008194', 'date': 'Fri Sep 27 16:18:16 +0000 2019', 'text': '@GaryTheBot Figure of speech'}, {'id': '1177618401159602176', 'date': 'Fri Sep 27 16:18:00 +0000 2019', 'text': 'Meanwhile, back in reality https://t.co/NrHHB9G81L'}, {'id': '1177618032719319040', 'date': 'Fri Sep 27 16:16:32 +0000 2019', 'text': '@DLoesch Media report was false'}, {'id': '1177387141116002304', 'date': 'Fri Sep 27 00:59:03 +0000 2019', 'text': '@Erdayastronaut @CorSPACE Raptor uses milled copper channels with an inconel jacket all the way down'}, {'id': '1177314408604680192', 'date': 'Thu Sep 26 20:10:03 +0000 2019', 'text': 'Three Raptors on a Starship https://t.co/UrRiD62EVk'}, {'id': '1177259561117372417', 'date': 'Thu Sep 26 16:32:06 +0000 2019', 'text': '@EvaFoxU @BrendanCogbill7 Always'}, {'id': '1177241843093753862', 'date': 'Thu Sep 26 15:21:42 +0000 2019', 'text': '@SpacePadreIsle @SpaceX Wow, its been 5 years since we started at Boca!'}, {'id': '1177237783489863681', 'date': 'Thu Sep 26 15:05:34 +0000 2019', 'text': '@BrendanCogbill7 About a week. Will include slight improvements, based on initial user feedback.'}, {'id': '1177236839406559232', 'date': 'Thu Sep 26 15:01:49 +0000 2019', 'text': '@justpaulinelol We look for rare corner case issues with a limited release US build, then expand internationally as https://t.co/Lqw9n2ZHvR'}, {'id': '1177235963405193216', 'date': 'Thu Sep 26 14:58:20 +0000 2019', 'text': '@GaryTheBot Gary, you rock!'}, {'id': '1177235772857970688', 'date': 'Thu Sep 26 14:57:34 +0000 2019', 'text': 'Tesla V10.0 just started rolling out to all US Tesla owners with FSD option &amp; advanced download selected in vehicle software settings'}, {'id': '1177101142980825093', 'date': 'Thu Sep 26 06:02:36 +0000 2019', 'text': '@moyo5150 More like https://t.co/f8BUlz0Xdo'}, {'id': '1177100271937212416', 'date': 'Thu Sep 26 05:59:08 +0000 2019', 'text': 'Anonymous bot swarms deserve a closer examination. If theyre evolving rapidly, somethings up.'}, {'id': '1177099375287271424', 'date': 'Thu Sep 26 05:55:35 +0000 2019', 'text': 'If advanced AI (beyond basic bots) hasnt been applied to manipulate social media, it wont be long before it is'}, {'id': '1177096273628778496', 'date': 'Thu Sep 26 05:43:15 +0000 2019', 'text': '@WallyWests Exactly'}, {'id': '1177090550916239360', 'date': 'Thu Sep 26 05:20:31 +0000 2019', 'text': '@Kristennetten @annerajb @Jennerator211 @Erdayastronaut That would be a nice touch'}, {'id': '1177085193007456256', 'date': 'Thu Sep 26 04:59:13 +0000 2019', 'text': '@flcnhvy @28delayslater @DJSnM This is quite a complex beast, but hopefully within a month or so'}, {'id': '1177084577128448000', 'date': 'Thu Sep 26 04:56:47 +0000 2019', 'text': '@28delayslater @DJSnM Haha awesome'}, {'id': '1177074975020220416', 'date': 'Thu Sep 26 04:18:37 +0000 2019', 'text': '@MarkChica @Kristennetten @TeslaGong @teslaownersSV @Teslatunity @Tesla We could probably do that'}, {'id': '1177070801121468418', 'date': 'Thu Sep 26 04:02:02 +0000 2019', 'text': '@Erdayastronaut 50m'}, {'id': '1177069611142901762', 'date': 'Thu Sep 26 03:57:18 +0000 2019', 'text': '@TeslaGong @teslaownersSV @Teslatunity @Tesla Not sure we could go quite that tight, but we can for sure allow much https://t.co/wolYBBZMS7'}, {'id': '1177067257106747392', 'date': 'Thu Sep 26 03:47:57 +0000 2019', 'text': '@annerajb @Jennerator211 @Erdayastronaut With rare exceptions, composites would make Starship heavier. They dont s https://t.co/OeeigvDiI8'}, {'id': '1177066483375058944', 'date': 'Thu Sep 26 03:44:53 +0000 2019', 'text': '@Jennerator211 @Erdayastronaut Mk1 ship is around 200 tons dry &amp; 1400 tons wet, but aiming for 120 by Mk4 or Mk5. T https://t.co/yTUpGx0Ppa'}, {'id': '1177064952429010945', 'date': 'Thu Sep 26 03:38:48 +0000 2019', 'text': '@teslaownersSV @Teslatunity @Tesla Absolutely'}, {'id': '1177062354577756160', 'date': 'Thu Sep 26 03:28:28 +0000 2019', 'text': '@Teslatunity @Tesla Youre welcome! Feature requests always appreciated.'}, {'id': '1177051800769138690', 'date': 'Thu Sep 26 02:46:32 +0000 2019', 'text': '@Erdayastronaut @kimitalvitie Six. Two windward, one under each fin &amp; two leeward. Provides redundancy for landing on unimproved surfaces.'}, {'id': '1177050841041125376', 'date': 'Thu Sep 26 02:42:43 +0000 2019', 'text': 'On the transporter https://t.co/CGNj1MlKAU'}, {'id': '1177049277375250434', 'date': 'Thu Sep 26 02:36:30 +0000 2019', 'text': 'Lifting lower section of Starship https://t.co/bfW17i469a'}, {'id': '1176936806928879616', 'date': 'Wed Sep 25 19:09:35 +0000 2019', 'text': '@justpaulinelol @EvaFoxU @katlinegrey https://t.co/IzVngWWY7G'}, {'id': '1176930839411642368', 'date': 'Wed Sep 25 18:45:53 +0000 2019', 'text': '@EvaFoxU @katlinegrey '}, {'id': '1176928113285971968', 'date': 'Wed Sep 25 18:35:03 +0000 2019', 'text': '@katlinegrey   '}, {'id': '1176737065624453125', 'date': 'Wed Sep 25 05:55:53 +0000 2019', 'text': '@levie Exactly'}, {'id': '1176596320775094273', 'date': 'Tue Sep 24 20:36:37 +0000 2019', 'text': '@tannerfox Impressive work by Chad &amp; Steve!'}, {'id': '1176589331567452161', 'date': 'Tue Sep 24 20:08:51 +0000 2019', 'text': '@SamNazarius @HarryStoltz1 @justpaulinelol @Teslarati Almost entirely C++, but we did have an (advanced for the tim https://t.co/0SbJqgtWsG'}, {'id': '1176588054670323713', 'date': 'Tue Sep 24 20:03:46 +0000 2019', 'text': '@HarryStoltz1 @justpaulinelol @Teslarati Turbo Pascal'}, {'id': '1176587698401947650', 'date': 'Tue Sep 24 20:02:21 +0000 2019', 'text': '@justpaulinelol @Teslarati Mostly C/C++, Python &amp; SQL'}, {'id': '1176586684751908872', 'date': 'Tue Sep 24 19:58:20 +0000 2019', 'text': '@flcnhvy @Erdayastronaut @Tesla It would be pretty embarrassing to use a non-Tesla battery!'}, {'id': '1176585904091275264', 'date': 'Tue Sep 24 19:55:14 +0000 2019', 'text': '@Teslarati This includes original engineering of new factory processes &amp; cars. Great engineers will only join if or https://t.co/zCAndlhS72'}]
#d = pd.DataFrame(d)

##changing the date format
date3 = []
for i in range(d.shape[0]):
    a = parse(d.date[i])
    date2 = a.strftime("%Y-%m-%d")
    date3.append(date2)
    
date3 = pd.DataFrame(date3)
d2 = pd.concat([d, date3], axis = 1)
d2.date = d2.iloc[:,[-1]]
#source = ["twitter"]*len(d2)
d2 = d2[['id', 'date', 'text']]
d2['source'] = "twitter"


client = language.LanguageServiceClient.from_service_account_json('/Users/jenniferwu/Downloads/second.json')

##calculating the score
score = []
for i in d2.text:
        text = i
        document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
        sentiment = client.analyze_sentiment(document=document).document_sentiment
        score.append(sentiment.score)
        
        
d2['score'] = score
d2['score_range'] = d2.score.apply(score_range)
output_1 = d2.groupby(['date'])['score_range'].agg(pd.Series.mode)
output = output_1.reset_index(level=['date'])
output['source']="twitter"
output['score_range'] = output.score_range.apply(np.mean)

In [230]:
output.head()

date  score_range   source
0  2019-09-24         0.00  twitter
1  2019-09-25         0.00  twitter
2  2019-09-26         0.00  twitter
3  2019-09-27         0.00  twitter
4  2019-09-28         0.06  twitter